In [1]:
# This function extends sage's number_of_roots_in_interval(a,b) function 
# so that it works with polynomials with non-squarefree polynomials
def numberOfRootsInInterval(f):
	R.<x>=PolynomialRing(QQ)
	ff=R(f)
	if ff.degree() < 1:
		return 0
	g = gcd(ff,ff.derivative())
	if g == 1:
		return ff.number_of_roots_in_interval()
	else:
		return numberOfRootsInInterval(ff/g) + numberOfRootsInInterval(g)


def getCoeffsWeaklyType2(d,clist):
	R.<x>=PolynomialRing(QQ)
	n=len(clist)-1
	coeffLists=[[] for i in range(d-n+1)]
	coeffLists[0]=[clist]
	for i in range(1,d-n+1):
		nn=len(coeffLists[i-1][0])-1
		for j in range(len(coeffLists[i-1])):
			coeffLists[i]+=getCoeffs(coeffLists[i-1][j],2^(nn),nn,d)
	return coeffLists[d-n]


def getCoeffsType2(d,clist):
	R.<x>=PolynomialRing(QQ)
	n=len(clist)-1
	coeffLists=[[] for i in range(d-n+1)]
	coeffLists[0]=[clist]
	for i in range(1,d-n+1):
		for j in range(len(coeffLists[i-1])):
			nn=len(coeffLists[i-1][j])-1
			coeffLists[i]+=getCoeffs(coeffLists[i-1][j],2^(nn+1),nn,d)
	return coeffLists[d-n]
	

def getCoeffs(clist,div,nn,d):
	R.<x>=PolynomialRing(QQ)
	S.<x,y>=PolynomialRing(QQ)
	T.<y>=PolynomialRing(QQ)
	f=R(list(reversed(clist)))
	f*=x^(d-nn)
	for j in range(1,d-nn):
		f=f.derivative(x)
		f/=j
	g = S(f + y)
	g = T(g.discriminant(x))
	roots = g.roots(RealField(1000))
	groots=[]
	for i in range(len(roots)):
	  groots+=[roots[i][0] for j in range(roots[i][1])]
	# groots.sort()
	if len(groots) != (T(g)).degree():
	  print clist
	  print div
	  print nn
	  print "possible error"
	pos=floor(nn/2)
	low=ceil(groots[pos-1])
	high=floor(groots[pos])
	ss=[]
	for j in range(ceil(low/div),floor(high/div)+1):
		if numberOfRootsInInterval(f+div*j) == nn+1:
			ss+=[clist+[div*j]]
	return ss
	
#input: ff and gg are real-rooted polynomials, ff is degree one more than gg
#output: True iff the roots of gg interlace with those of ff
def interlace(ff,gg):
	R.<x>=PolynomialRing(QQ)
	f=R(ff)
	g=R(gg)
	d=gcd(f,g)
	f/=d
	g/=d
	rootsg = (R(g)).roots(RealField(1000))
	groots=[]
	for i in range(len(rootsg)):
		groots+=[rootsg[i][0] for j in range(rootsg[i][1])]
	# groots.sort()
	rootsf = (R(f)).roots(RealField(1000))
	froots=[]
	for i in range(len(rootsf)):
		froots+=[rootsf[i][0] for j in range(rootsf[i][1])]
		# groots.sort()
	for i in range(len(groots)):
		if groots[i] < froots[i]:
			return(False)
		if groots[i] > froots[i+1]:
			return(False)
	return(True)

def listInterlaceReduce(l,f):
	R.<x>=PolynomialRing(QQ)
	ll=[];
	for i in range(len(l)):
		p = R(list(reversed(l[i])))
		if interlace(f,p):
			ll+=[l[i]]
	return(ll);
  
def isType2(f):
	v=f.coefficients(sparse=False)
	v.reverse()
	for i in range(len(v)):
		if v[i]%2^i != 0:
			return False
	return True
  
def listPowerReduce(l,f):
	R.<x>=PolynomialRing(QQ)
	ll=[];
	for i in range(len(l)):
		p = R(list(reversed(l[i])))
		pp=R(f)*p
		if isType2(pp(x-1)):
			ll+=[l[i]]
	return ll
  
def randomSeidel(n):
	m=matrix(n,n)
	for i in range(n):
		for j in range(i+1,n):
			r=ZZ.random_element(0,2)
			r=1-2*r
			m[i,j]=r
			m[j,i]=r
	return m
  
def SeidelCharPolysMod(n,k):
	ll=Set([])
	R.<x> = IntegerModRing(2^k)[]
	if n % 2 == 1:
		ub=2^(binomial(k-2,2)+1)
	else:
		2^(binomial(k-2,2))
	while len(ll) < ub:
		m=randomSeidel(n)
		ll+=Set([R(m.charpoly())])
	return ll
  
def listPowerReduceOdd(lst,f,n,k):
	R.<x>=IntegerModRing(2^k)[]
	ll=[]
	scc=SeidelCharPolysMod(n,k)
	for i in range(len(lst)):
		p = R(list(reversed(lst[i])))
		pp=R(f)*p
		if pp in scc:
			ll+=[lst[i]]
	return ll

#Use precomputed charpoly classes
def listPowerReduceOddPC(lst,f,scc,k):
	R.<x>=IntegerModRing(2^k)[]
	ll=[]
	for i in range(len(lst)):
		p = R(list(reversed(lst[i])))
		pp=R(f)*p
		if pp in scc:
			ll+=[lst[i]]
	return ll

def isAtLeastZero(l):
	for i in range(len(l)):
		if l[i] < 0:
			return False
	return True

import time  
R.<x>=IntegerModRing(2^7)[]

In [2]:
#precompute Seidel char polys classes n=75, e=7
#scc757=SeidelCharPolysMod(75,7)
load("mcP757.sage")

In [3]:
#precompute Seidel char polys classes n=73, e=7
#scc737=SeidelCharPolysMod(73,7)
load("mcP737.sage")

In [4]:
#check Lemma 5.3
def lem53Polynomials():
	R.<x>=PolynomialRing(QQ)
	ll2=getCoeffsWeaklyType2(7,[1,-107,4896])
	llshift2=[R(list(reversed(l)))(x+1) for l in ll2]
	lllist2=[list(reversed(l.coefficients(sparse=False))) for l in llshift2]
	out=listPowerReduceOddPC(lllist2,(x+5)^56*(x-15)^12,scc757,7)
	return out

def checkLemma53():
	R.<x>=PolynomialRing(QQ)
	print "--- Candidates for char poly corresponding to 75 lines ---"
	print ""
	tic = time.time()
	cand75red=lem53Polynomials()
	toc = time.time()
	print "Time taken to compute char poly candidates: " + str(toc-tic) + " seconds"
	print ""
	print "Candidates for char poly:"
	for i in range(len(cand75red)):
		ff=R(list(reversed(cand75red[i])))*(x+5)^56*(x-15)^12
		ms=R(ff/gcd(ff,ff.derivative()))
		print i+1, factor(ff)
	print ""
	cand75=[cand75red[i] for i in [1,4,2,3,5,6]]
	FarkasCertificates=[
	[18413,257776,0,0,1876556160],
	[9645,135016,0,0,888359172],
	[-1033,-3203, 0, 0, 0], 
	[16674,123379, 523203, 0, 0, 0],
	[5571, 72416, 0, 0, 417044628],
	[-1337,-3999, 0, 0, 0]
	]
	bag=[]
	for i in range(len(cand75)):
		ff=R(list(reversed(cand75[i])))*(x+5)^56*(x-15)^12
		ms=R(ff/gcd(ff,ff.derivative()))
		g=R(ff.derivative()/gcd(ff,ff.derivative()))
		deg=ms.degree()
		f=ms+74*x^(deg-2)
		coeffs=list(reversed(f.coefficients(sparse=False)))
		gf=x^(deg-3)*R([coeffs[2],coeffs[1],coeffs[0]])
		ggf=gf(x-1)
		clist=list(reversed(ggf.coefficients(sparse=False)))
		print "------- Infeasibility of " + str(factor(ff)) + " -------"
		print ""
		tic = time.time()
		ll=getCoeffsType2(deg-1,[clist[0],clist[1],clist[2]])
		llpol=[R(list(reversed(l)))(x+1) for l in ll]
		ll2=[list(reversed(l.coefficients(sparse=False))) for l in llpol]
		bag+=[[factor(ff),listInterlaceReduce(ll2,ms)]]
		toc = time.time()
		print "Time taken to compute interlacing char polys: " + str(toc-tic) + " seconds"
		print ""
		deg=ms.degree()
		numrows=len(bag[i][1])
		m=matrix(numrows,deg,[bag[i][1][j] for j in range(numrows)])
		gg=vector(list(reversed(g.coefficients(sparse=False))))
		print "Coefficient matrix A for interlacing char polys:"
		print m
		print ""
		mm=vector(list(reversed(FarkasCertificates[i])))
		print "Certificate of infeasibility c:"
		print "c = " + str(mm)
		ac=mm*m.transpose()
		print "Ac >= 0: " + str(isAtLeastZero(ac))
		print "Ac = " + str(ac)
		innerprod=sum([gg[j]*mm[j] for j in range(deg)])
		print "<g,c> < 0: " + str(innerprod < 0)
		print "<g,c> = " + str(innerprod)
		print ""



# check Lemma 5.4
# (x + 5)^56*(x-13)^4*(x-15)^14*(x-18)
def lem54InterlacingCharacteristicPolynomials():
	R.<x>=PolynomialRing(QQ)
	ll2=getCoeffsType2(3,[1,-44,628])
	llshift2=[R(list(reversed(l)))(x+1) for l in ll2]
	lllist2=[list(reversed(l.coefficients(sparse=False))) for l in llshift2]
	llred2=listInterlaceReduce(lllist2,(x+5)*(x-13)*(x-15)*(x-18))
	return listPowerReduce(llred2,(x+5)^55*(x-13)^3*(x-15)^13)

def checkLemma54Part1():
	R.<x>=PolynomialRing(QQ)
	tic = time.time()
	llSeidelred=lem54InterlacingCharacteristicPolynomials()
	toc = time.time()
	print "Time taken to compute interlacing char polys: " + str(toc-tic) + " seconds"
	print ""
	print "Interlacing characteristic polynomials:"
	for i in range(len(llSeidelred)):
		ff=R(list(reversed(llSeidelred[i])))*(x+5)^55*(x-13)^3*(x-15)^13
		ms=R(ff/gcd(ff,ff.derivative()))
		print i+1, factor(ff)
	print ""
	numrows=len(llSeidelred)
	m=matrix(numrows,4,[llSeidelred[j] for j in range(numrows)])
	mults=vector([56,4,14,1])
	froots=[-5,13,15,18]
	f=prod([(x-r) for r in froots])
	fbas=[R(f/(x-r)) for r in froots]
	fbaslist=[list(reversed(l.coefficients(sparse=False))) for l in fbas]
	fbasm=matrix(4,4,[fbaslist[j] for j in range(4)])
	bb=mults*fbasm
	#submatrix of m without f1
	mf1=m[range(1,numrows),range(4)]
	print "Matrix A' corresponding to interlacing polynomials without f1:" 
	print mf1
	print "" 
	#Farkas certificate 
	fc1=vector(list(reversed([151,0,0,349511])))
	print "Certificate of warranty c:"
	print "c = " + str(fc1)  
	ac = fc1*mf1.transpose()
	print "Ac >= 0: " + str(isAtLeastZero(ac))
	print "Ac = " + str(ac)
	innerprod = sum([fc1[j]*bb[j] for j in range(4)])
	print "<g,c> < 0: " + str(innerprod < 0)
	print "<g,c> = " + str(innerprod)

def lem54InterlacingCharacteristicPolynomials2():
	R.<x>=PolynomialRing(QQ)
	ll2=getCoeffsWeaklyType2(5,[1,-69,1880])
	llshift2=[R(list(reversed(l)))(x+1) for l in ll2]
	lllist2=[list(reversed(l.coefficients(sparse=False))) for l in llshift2]
	llred2=listInterlaceReduce(lllist2,(x+5)*(x-13)*(x-15)*(x^3-41*x^2+543*x-2319))
	return listPowerReduceOddPC(llred2,(x+5)^54*(x-13)^2*(x-15)^12,scc737,7)

def checkLemma54Part2():
	R.<x>=PolynomialRing(QQ)
	tic = time.time()
	llSeidelred=lem54InterlacingCharacteristicPolynomials2()
	toc = time.time()
	print "Time taken to compute interlacing char polys: " + str(toc-tic) + " seconds"
	print ""
	print "Interlacing characteristic polynomials:"
	for i in range(len(llSeidelred)):
		ff=R(list(reversed(llSeidelred[i])))*(x+5)^54*(x-13)^2*(x-15)^12
		ms=R(ff/gcd(ff,ff.derivative()))
		print i+1, factor(ff)
	print ""
	numrows=len(llSeidelred)
	m=matrix(numrows,6,[llSeidelred[j] for j in range(numrows)])
	mults=vector([55,3,13,1,1,1])
	N.<al>=NumberField(x^3-41*x^2 + 543*x -2319)
	N1.<z>=PolynomialRing(N)
	N2.<be>=NumberField(z^2 + (al - 41)*z + al^2 - 41*al + 543)
	NN.<zz>=PolynomialRing(N2)
	froots=[-5,13,15,al,be,41-al-be]
	f=prod([(z-r) for r in froots])
	fbas=[NN(f/(z-r)) for r in froots]
	fbaslist=[list(reversed(l.coefficients(sparse=False))) for l in fbas]
	fbasm=matrix(6,6,[fbaslist[j] for j in range(6)])
	bb=mults*fbasm
	print "Coefficient matrix A for interlacing char polys:"
	print m
	print ""
	#Farkas certificate 
	fc1=vector([55783250920,0,0,4439925,330721,24804]) 
	print "Certificate of infeasibility c:"
	print "c = " + str(fc1)  
	ac = fc1*m.transpose()
	print "Ac >= 0: " + str(isAtLeastZero(ac))
	print "Ac = " + str(ac)
	innerprod = sum([fc1[j]*bb[j] for j in range(6)])
	print "<g,c> < 0: " + str(RR(innerprod) < 0)
	print "<g,c> = " + str(innerprod)

def checkLemma54():
	R.<x>=PolynomialRing(QQ)
	print "--- Infeasibility of " + str(factor((x + 5)^56*(x-13)^4*(x-15)^14*(x-18))) + "  ---"
	print ""
	checkLemma54Part1()
	print ""
	print "--- Infeasibility of f1 ---"
	print ""
	checkLemma54Part2()

In [5]:
#precompute Seidel char polys classes n=95, e=7
#scc957=SeidelCharPolysMod(95,7)
load("mcP957.sage")

In [6]:
#precompute Seidel char polys classes n=93, e=7
#scc937=SeidelCharPolysMod(93,7)
load("mcP937.sage")

In [7]:
# check Lemma 5.6
def lem56Polynomials():
	R.<x>=PolynomialRing(QQ)
	ll2=getCoeffsWeaklyType2(7,[1,-135,7800])
	llshift2=[R(list(reversed(l)))(x+1) for l in ll2]
	lllist2=[list(reversed(l.coefficients(sparse=False))) for l in llshift2]
	return listPowerReduceOddPC(lllist2,(x+5)^75*(x-19)^13,scc957,7)

def checkLemma56():
	R.<x>=PolynomialRing(QQ)
	print "--- Candidates for char poly corresponding to 95 lines ---"
	print ""
	tic = time.time()
	cand95red=lem56Polynomials()
	toc = time.time()
	print "Time taken to compute char poly candidates: " + str(toc-tic) + " seconds"
	print ""
	print "Candidates for char poly:"
	for i in range(len(cand95red)):
		ff=R(list(reversed(cand95red[i])))*(x+5)^75*(x-19)^13
		ms=R(ff/gcd(ff,ff.derivative()))
		print i+1, factor(ff)
	print ""
	cand95=[cand95red[i] for i in [1,4,2,3,5,6]]
	FarkasCertificates=[
	[49420,889549,0,0,14218410144],
	[26657,479808,0,0,7099110312],
	[-1095,-4529, 0, 0, 0], 
	[8956, 88657, 497653, 0, 0, 0],
	[7845, 133358, 0, 0, 1784197530],
	[-2587,-10412, 0, 0, 0]
	]
	bag=[]
	for i in range(len(cand95)):
		ff=R(list(reversed(cand95[i])))*(x+5)^75*(x-19)^13
		ms=R(ff/gcd(ff,ff.derivative()))
		g=R(ff.derivative()/gcd(ff,ff.derivative()))
		deg=ms.degree()
		f=ms+94*x^(deg-2)
		coeffs=list(reversed(f.coefficients(sparse=False)))
		gf=x^(deg-3)*R([coeffs[2],coeffs[1],coeffs[0]])
		ggf=gf(x-1)
		clist=list(reversed(ggf.coefficients(sparse=False)))
		print "------- Infeasibility of " + str(factor(ff)) + " -------"
		print ""
		tic = time.time()
		ll=getCoeffsType2(deg-1,[clist[0],clist[1],clist[2]])
		llpol=[R(list(reversed(l)))(x+1) for l in ll]
		ll2=[list(reversed(l.coefficients(sparse=False))) for l in llpol]
		bag+=[[factor(ff),listInterlaceReduce(ll2,ms)]]
		toc = time.time()
		print "Time taken to compute interlacing char polys: " + str(toc-tic) + " seconds"
		print ""
		deg=ms.degree()
		numrows=len(bag[i][1])
		m=matrix(numrows,deg,[bag[i][1][j] for j in range(numrows)])
		gg=vector(list(reversed(g.coefficients(sparse=False))))
		print "Coefficient matrix A for interlacing char polys:"
		print m
		print ""
		mm=vector(list(reversed(FarkasCertificates[i])))
		print "Certificate of infeasibility c:"
		print "c = " + str(mm)
		ac=mm*m.transpose()
		print "Ac >= 0: " + str(isAtLeastZero(ac))
		print "Ac = " + str(ac)
		innerprod=sum([gg[j]*mm[j] for j in range(deg)])
		print "<g,c> < 0: " + str(innerprod < 0)
		print "<g,c> = " + str(innerprod)
		print ""


# check Lemma 5.7
# (x + 5)^56*(x-13)^4*(x-15)^14*(x-18)
def lem57InterlacingCharacteristicPolynomials():
	R.<x>=PolynomialRing(QQ)
	ll2=getCoeffsType2(3,[1,-56,1028])
	llshift2=[R(list(reversed(l)))(x+1) for l in ll2]
	lllist2=[list(reversed(l.coefficients(sparse=False))) for l in llshift2]
	llred2=listInterlaceReduce(lllist2,(x+5)*(x-17)*(x-19)*(x-22))
	return listPowerReduce(llred2,(x+5)^74*(x-17)^3*(x-19)^14)

def checkLemma57Part1():
	R.<x>=PolynomialRing(QQ)
	tic = time.time()
	llSeidelred=lem57InterlacingCharacteristicPolynomials()
	toc = time.time()
	print "Time taken to compute interlacing char polys: " + str(toc-tic) + " seconds"
	print ""
	print "Interlacing characteristic polynomials:"
	for i in range(len(llSeidelred)):
		ff=R(list(reversed(llSeidelred[i])))*(x+5)^74*(x-17)^3*(x-19)^14
		ms=R(ff/gcd(ff,ff.derivative()))
		print i+1, factor(ff)
	print ""
	numrows=len(llSeidelred)
	m=matrix(numrows,4,[llSeidelred[j] for j in range(numrows)])
	mults=vector([75,4,15,1])
	froots=[-5,17,19,22]
	f=prod([(x-r) for r in froots])
	fbas=[R(f/(x-r)) for r in froots]
	fbaslist=[list(reversed(l.coefficients(sparse=False))) for l in fbas]
	fbasm=matrix(4,4,[fbaslist[j] for j in range(4)])
	bb=mults*fbasm
	#submatrix of m without f1
	mf1=m[range(1,numrows),range(4)]
	print "Matrix A' corresponding to interlacing polynomials without f1:" 
	print mf1
	print "" 
	#Farkas certificate 
	fc1=vector(list(reversed([917,0,0,4776043])))
	print "Certificate of warranty c:"
	print "c = " + str(fc1)  
	ac = fc1*mf1.transpose()
	print "Ac >= 0: " + str(isAtLeastZero(ac))
	print "Ac = " + str(ac)
	innerprod = sum([fc1[j]*bb[j] for j in range(4)])
	print "<g,c> < 0: " + str(innerprod < 0)
	print "<g,c> = " + str(innerprod)


def lem57InterlacingCharacteristicPolynomials2():
	R.<x>=PolynomialRing(QQ)
	ll2=getCoeffsWeaklyType2(5,[1,-89,3144])
	llshift2=[R(list(reversed(l)))(x+1) for l in ll2]
	lllist2=[list(reversed(l.coefficients(sparse=False))) for l in llshift2]
	llred2=listInterlaceReduce(lllist2,(x+5)*(x-17)*(x-19)*(x^3-53*x^2+919*x-5211))
	return listPowerReduceOddPC(llred2,(x+5)^73*(x-17)^2*(x-19)^13,scc937,7)

def checkLemma57Part2():
	R.<x>=PolynomialRing(QQ)
	tic = time.time()
	llSeidelred=lem57InterlacingCharacteristicPolynomials2()
	toc = time.time()
	print "Time taken to compute interlacing char polys: " + str(toc-tic) + " seconds"
	print ""
	print "Interlacing characteristic polynomials:"
	for i in range(len(llSeidelred)):
		ff=R(list(reversed(llSeidelred[i])))*(x+5)^74*(x-17)^2*(x-19)^13
		ms=R(ff/gcd(ff,ff.derivative()))
		print i+1, factor(ff)
	print ""
	numrows=len(llSeidelred)
	m=matrix(numrows,6,[llSeidelred[j] for j in range(numrows)])
	mults=vector([74,3,14,1,1,1])
	N.<al>=NumberField(x^3-53*x^2 + 919*x -5211)
	N1.<z>=PolynomialRing(N)
	N2.<be>=NumberField(z^2 + (al - 53)*z + al^2 - 53*al + 919)
	NN.<zz>=PolynomialRing(N2)
	froots=[-5,17,19,al,be,53-al-be]
	f=prod([(z-r) for r in froots])
	fbas=[NN(f/(z-r)) for r in froots]
	fbaslist=[list(reversed(l.coefficients(sparse=False))) for l in fbas]
	fbasm=matrix(6,6,[fbaslist[j] for j in range(6)])
	bb=mults*fbasm
	print "Coefficient matrix A for interlacing char polys:"
	print m
	print ""
	#Farkas certificate 
	fc1=vector(list(reversed([265131,4595613,79981334,0,0,2282735504746])))  
	print "Certificate of infeasibility c:"
	print "c = " + str(fc1)  
	ac = fc1*m.transpose()
	print "Ac >= 0: " + str(isAtLeastZero(ac))
	print "Ac = " + str(ac)
	innerprod = sum([fc1[j]*bb[j] for j in range(6)])
	print "<g,c> < 0: " + str(RR(innerprod) < 0)
	print "<g,c> = " + str(innerprod)

def checkLemma57():
	R.<x>=PolynomialRing(QQ)
	print "--- Infeasibility of " + str(factor((x + 5)^75*(x-17)^4*(x-19)^15*(x-22))) + "  ---"
	print ""
	checkLemma57Part1()
	print ""
	print "--- Infeasibility of f1 ---"
	print ""
	checkLemma57Part2()

In [8]:
#precompute Seidel char polys classes n=29, e=7
#scc297=SeidelCharPolysMod(29,7)
load("mcP297.sage")

In [9]:
#precompute Seidel char polys classes n=27, e=7
#scc277=SeidelCharPolysMod(27,7)
load("mcP277.sage")

In [10]:
# get candidates for 29 lines
def lem62Polynomials():
	R.<x>=PolynomialRing(QQ)
	cand29=getCoeffsWeaklyType2(10,[1,-65,1884])
	cand29shift=[R(list(reversed(l)))(x+1) for l in cand29]
	cand29list=[list(reversed(l.coefficients(sparse=False))) for l in cand29shift]
	return listPowerReduceOddPC(cand29list,(x+5)^15*(x-5)^4,scc297,7)

def checkLemma62():
	R.<x>=PolynomialRing(QQ)
	print "--- Candidates for char poly corresponding to 29 lines ---"
	print ""
	tic = time.time()
	cand29red=lem62Polynomials()
	toc = time.time()
	print "Time taken to compute char poly candidates: " + str(toc-tic) + " seconds"
	print ""
	print "Candidates for char poly:"
	for i in range(len(cand29red)):
		ff=R(list(reversed(cand29red[i])))*(x+5)^15*(x-5)^4
		ms=R(ff/gcd(ff,ff.derivative()))
		print i+1, factor(ff)
	print ""
	cand29=[cand29red[i] for i in [0,30,24,22,4,2,20,28,29,14,25,27,12,9,3,17,15,13,23,19,26,8,11,21,18]]
	FarkasCertificates=[
	[44,0,0,423], 
	[-126,-631,0,0,-30576], 
	[0,-114,0,0,-11155], 
	[51,458,0,0,87568],
	[-366,-2563,0,0,-544242],
	[-2820, -12969, -59189, 0, 0, -34793172], 
	[361,1084,4712,0,0,1656424], 
	[0,0,971,0,0,328314], 
	[76,75,67,0,0,0],
	[112,395,479,0,0,0], 
	[184,1105,6816,0,0,3452532], 
	[-52,-173,-157,0,0,0], 
	[-217, -1528, -11608, 0, 0, -6439670],
	[-71,-265,-339,0,0,0], 
	[0,676,3743,8141,0,0,0], 
	[-1018,0,8413,20451,0,0,0], 
	[0,0,3,-3784,0,0,-4921960], 
	[-2015, -14106, -98738, -690887, 0, 0, -899357723],
	[-242,0,-17,0,0,0,0], 
	[238,978,3367,6028,0,0,0], 
	[174,616,1795,2863,0,0,0], 
	[425, 1747, 6111, 11531, 0, 0, 0], 
	[4, -500, -3625, -25940, -184580, 0, 0, -391700060], 
	[0, 2250, 25343, 212711, 1568930, 0, 0, 3329450510],
	[-2001, -7066, -19411, -27929, 0, 0, 0, 0]]
	bag=[]
	for i in range(len(cand29)):
		ff=R(list(reversed(cand29[i])))*(x+5)^15*(x-5)^4
		ms=R(ff/gcd(ff,ff.derivative()))
		g=R(ff.derivative()//gcd(ff,ff.derivative()))
		deg=ms.degree()
		f=ms+28*x^(deg-2)
		coeffs=list(reversed(f.coefficients(sparse=False)))
		gf=x^(deg-3)*R([coeffs[2],coeffs[1],coeffs[0]])
		ggf=gf(x-1)
		clist=list(reversed(ggf.coefficients(sparse=False)))
		print "------- Infeasibility of " + str(factor(ff)) + " -------"
		print ""
		tic = time.time()
		ll=getCoeffsType2(deg-1,[clist[0],clist[1],clist[2]])
		llpol=[R(list(reversed(l)))(x+1) for l in ll]
		ll2=[list(reversed(l.coefficients(sparse=False))) for l in llpol]
		bag+=[[factor(ff),listInterlaceReduce(ll2,ms)]]
		toc = time.time()
		print "Time taken to compute interlacing char polys: " + str(toc-tic) + " seconds"
		print ""
		deg=ms.degree()
		numrows=len(bag[i][1])
		m=matrix(numrows,deg,[bag[i][1][j] for j in range(numrows)])
		gg=vector(list(reversed(g.coefficients(sparse=False))))
		print "Coefficient matrix A for interlacing char polys:"
		print m
		print ""
		mm=vector(list(reversed(FarkasCertificates[i])))
		print "Certificate of infeasibility c:"
		print "c = " + str(mm)
		ac=mm*m.transpose()
		print "Ac >= 0: " + str(isAtLeastZero(ac))
		print "Ac = " + str(ac)
		innerprod=sum([gg[j]*mm[j] for j in range(deg)])
		print "<g,c> < 0: " + str(innerprod < 0)
		print "<g,c> = " + str(innerprod)
		print ""

# check Lemma 6.3
# (x + 5)^15*(x-5)^10*(x -7)^2*(x^2 - 11*x + 16)
def lem63InterlacingCharacteristicPolynomials():
	R.<x>=PolynomialRing(QQ)
	ll=getCoeffsType2(4,[1,-22,156])
	llshift=[R(list(reversed(l)))(x+1) for l in ll]
	lllist=[list(reversed(l.coefficients(sparse=False))) for l in llshift]
	llred=listInterlaceReduce(lllist,(x + 5)*(x-5)*(x -7)*(x^2 - 11*x + 16))
	return listPowerReduce(llred,(x+5)^14*(x-5)^9*(x-7))

def checkLemma63():
	R.<x>=PolynomialRing(QQ)
	print "--- Infeasibility of " + str(factor((x + 5)^15*(x-5)^10*(x -7)^2*(x^2 - 11*x + 16))) + "  ---"
	print ""
	tic = time.time()
	llSeidelred=lem63InterlacingCharacteristicPolynomials()
	toc = time.time()
	print "Time taken to compute interlacing char polys: " + str(toc-tic) + " seconds"
	print ""
	numrows=len(llSeidelred)
	m=matrix(numrows,5,[llSeidelred[j] for j in range(numrows)])
	mults=vector([1,1,15,10,2])
	N.<sqrt57>=NumberField(x^2 - 57)
	aln=(11+sqrt(11^2-4*16))/2
	NN.<z>=PolynomialRing(N)
	al=(11-sqrt57)/2
	froots=[al,11-al,-5,5,7]
	f=prod([(z-r) for r in froots])
	fbas=[NN(f/(z-r)) for r in froots]
	fbaslist=[list(reversed(l.coefficients(sparse=False))) for l in fbas]
	fbasm=matrix(5,5,[fbaslist[j] for j in range(5)])
	bb=mults*fbasm
	#submatrix of m without f1
	mf1=m[range(1,31),range(5)] 
	print "Matrix A' corresponding to interlacing polynomials without f1:" 
	print mf1
	print ""
	#Farkas certificate 
	fc1=vector([3936,0,0,29,0])  
	print "Certificate of warranty c for f1:"
	print "c = " + str(fc1)
	ac=fc1*mf1.transpose()
	print "A'c >= 0: " + str(isAtLeastZero(ac))
	print "A'c = " + str(ac)
	innerprod=sum([fc1[j]*bb[j] for j in range(5)])
	print "<g,c> < 0: " + str(innerprod < 0)
	print "<g,c> = " + str(innerprod)
	print ""
	#submatrix of m without f2:
	mf2=m[[0,1,2,3]+range(5,31),range(5)]
	print "Matrix A' corresponding to interlacing polynomials without f2:" 
	print mf2
	print ""
	#Farkas certificate:
	fc2=vector([-333696,0,0,-2459,-492])
	print "Certificate of warranty c for f2:"
	print "c = " + str(fc2)   
	ac = fc2*mf2.transpose()
	print "A'c >= 0: " + str(isAtLeastZero(ac))
	print "A'c = " + str(ac)
	innerproduct=sum([fc2[j]*bb[j] for j in range(5)])
	print "<g,c> < 0: " + str(innerprod < 0)
	print "<g,c> = " + str(innerprod)
	print ""
	f1=vector([m[0,j] for j in range(5)])
	f2=vector([m[4,j] for j in range(5)])
	af1=f1*fbasm^-1
	af2=f2*fbasm^-1
	print "squared angles for f1:"
	print af1
	print "squared angles for f2:"
	print af2
	print "--- checking for compatibility of f1 and f2 ---"
	p=(z^2-25)*(z-7)
	L1.<be1>=NumberField(z^2-af2[0]*af1[0])
	LL1.<w2>=PolynomialRing(L1)
	if (w2^2-af2[1]*af1[1]).is_irreducible():
		print "alpha_{2,f1}*alpha_{2,f2} is not in N(alpha_{1,f1}*alpha_{1,f2})"
	print "--- end of compatibility check ---"
	print ""

# check Lemma 6.4
# (x + 5)^15*(x-3)*(x-5)^9*(x -7)^2*(x^2 - 13*x + 32)
def lem64InterlacingCharacteristicPolynomials():
	R.<x>=PolynomialRing(QQ)
	ll=getCoeffsType2(5,[1,-28,288])
	llshift=[R(list(reversed(l)))(x+1) for l in ll]
	lllist=[list(reversed(l.coefficients(sparse=False))) for l in llshift]
	llred=listInterlaceReduce(lllist,(x + 5)*(x-3)*(x-5)*(x -7)*(x^2 - 13*x + 32))
	return listPowerReduce(llred,(x+5)^14*(x-5)^8*(x-7))

def checkLemma64():
	R.<x>=PolynomialRing(QQ)
	print "--- Infeasibility of " + str(factor((x + 5)^15*(x-3)*(x-5)^9*(x -7)^2*(x^2 - 13*x + 32))) + "  ---"
	print ""
	tic = time.time()
	llSeidelred=lem64InterlacingCharacteristicPolynomials()
	toc = time.time()
	print "Time taken to compute interlacing char polys: " + str(toc-tic) + " seconds"
	print ""
	numrows=len(llSeidelred)
	m=matrix(numrows,6,[llSeidelred[j] for j in range(numrows)])
	mults=vector([1,1,1,15,9,2])
	N.<sqrt41>=NumberField(x^2 - 41)
	aln=(13+sqrt(13^2-4*32))/2
	NN.<z>=PolynomialRing(N)
	al=(13-sqrt41)/2
	froots=[3,al,13-al,-5,5,7]
	f=prod([(z-r) for r in froots])
	fbas=[NN(f/(z-r)) for r in froots]
	fbaslist=[list(reversed(l.coefficients(sparse=False))) for l in fbas]
	fbasm=matrix(6,6,[fbaslist[j] for j in range(6)])
	bb=mults*fbasm
	#submatrix of m without f1
	mf1=m[range(1,numrows),range(6)]
	print "Matrix A' corresponding to interlacing polynomials without f1:" 
	print mf1
	print ""
	#Farkas certificate 
	fc1=vector([0,0,0,-845,-703,-235])
	print "Certificate of warranty c for f1:"
	print "c = " + str(fc1)
	ac = fc1*mf1.transpose()
	print "A'c >= 0: " + str(isAtLeastZero(ac))
	print "A'c = " + str(ac)
	innerprod = sum([fc1[j]*bb[j] for j in range(6)])
	print "<g,c> < 0: " + str(innerprod < 0)
	print "<g,c> = " + str(innerprod)
	print ""
	#submatrix of m without f2:
	mf2=m[[0,1,2,3]+range(5,numrows),range(6)]
	print "Matrix A' corresponding to interlacing polynomials without f2:" 
	print mf2
	print ""
	#Farkas certificate:
	fc2=vector([-2729736,0,0,-6261,-1593,-433])
	print "Certificate of warranty c for f2:"
	print "c = " + str(fc2)  
	ac = fc2*mf2.transpose()
	print "A'c >= 0: " + str(isAtLeastZero(ac))
	print "A'c = " + str(ac)
	innerprod = sum([fc2[j]*bb[j] for j in range(6)])
	print "<g,c> < 0: " + str(innerprod < 0)
	print "<g,c> = " + str(innerprod)
	print ""
	f1=vector([m[0,j] for j in range(6)])
	f2=vector([m[4,j] for j in range(6)])
	af1=f1*fbasm^-1
	af2=f2*fbasm^-1
	print "squared angles for f1:"
	print af1
	print "squared angles for f2:"
	print af2
	
	print "--- checking for compatibility of f1 and f2 ---"
	p=(z^2-25)*(z-3)*(z-7)
	L1.<be1>=NumberField(z^2-af2[1]*af1[1])
	LL1.<w2>=PolynomialRing(L1)
	if (w2^2-af2[2]*af1[2]).is_irreducible():
		print "alpha_{3,f1}*alpha_{3,f2} is not in N(alpha_{2,f1}*alpha_{2,f2})"
	print "--- end of compatibility check ---"
	print ""

# check Lemma 6.5
# (x + 5)^15*(x-5)^10*(x -7)*(x^3-18*x^2 +93*x - 128)
def lem65InterlacingCharacteristicPolynomials():
	R.<x>=PolynomialRing(QQ)
	ll=getCoeffsType2(5,[1,-30,332])
	llshift=[R(list(reversed(l)))(x+1) for l in ll]
	lllist=[list(reversed(l.coefficients(sparse=False))) for l in llshift]
	llred=listInterlaceReduce(lllist,(x + 5)*(x-5)*(x -7)*(x^3-18*x^2 +93*x - 128))
	return listPowerReduce(llred,(x+5)^14*(x-5)^9)

def checkLemma65():
	R.<x>=PolynomialRing(QQ)
	print "--- Infeasibility of " + str(factor((x + 5)^15*(x-5)^10*(x -7)*(x^3-18*x^2 +93*x - 128))) + "  ---"
	print ""
	tic = time.time()
	llSeidelred=lem65InterlacingCharacteristicPolynomials()
	toc = time.time()
	print "Time taken to compute interlacing char polys: " + str(toc-tic) + " seconds"
	print ""
	numrows=len(llSeidelred)
	m=matrix(numrows,6,[llSeidelred[j] for j in range(numrows)])
	mults=vector([1,1,1,1,15,10])
	N.<al>=NumberField(x^3-18*x^2 + 93*x -128)
	N1.<z>=PolynomialRing(N)
	N2.<be>=NumberField(z^2 + (al - 18)*z + al^2 - 18*al + 93)
	NN.<zz>=PolynomialRing(N2)
	froots=[al,be,18-al-be,7,-5,5]
	f=prod([(zz-r) for r in froots])
	fbas=[NN(f/(zz-r)) for r in froots]
	fbaslist=[list(reversed(l.coefficients(sparse=False))) for l in fbas]
	fbasm=matrix(6,6,[fbaslist[j] for j in range(6)])
	bb=mults*fbasm
	#submatrix of m without f1
	mf1=m[[0,1]+range(3,numrows),range(6)]  
	print "Matrix A' corresponding to interlacing polynomials without f1:" 
	print mf1
	print ""
	#Farkas certificate 
	fc1=vector([53248032,0,0,80553,12395,2066])
	print "Certificate of warranty c for f1:"
	print "c = " + str(fc1)
	ac = fc1*mf1.transpose()
	print "A'c >= 0: " + str(isAtLeastZero(ac))
	print "A'c = " + str(ac)
	innerprod = sum([fc1[j]*bb[j] for j in range(6)])
	print "<g,c> < 0: " + str(RR(innerprod) < 0)
	print "<g,c> = " + str(innerprod)
	print ""
	#submatrix of m without f2:
	mf2=m[range(12)+range(13,numrows),range(6)]
	print "Matrix A' corresponding to interlacing polynomials without f2:" 
	print mf2
	print ""
	#Farkas certificate:
	fc2=vector([0,0,0,416,329,101])
	print "Certificate of warranty c for f2:"
	print "c = " + str(fc2)  
	ac = fc2*mf2.transpose()
	print "A'c >= 0: " + str(isAtLeastZero(ac))
	print "A'c = " + str(ac)
	innerprod = sum([fc2[j]*bb[j] for j in range(6)])
	print "<g,c> < 0: " + str(RR(innerprod) < 0)
	print "<g,c> = " + str(innerprod)
	print ""
	f1=vector([m[2,j] for j in range(6)])
	f2=vector([m[12,j] for j in range(6)])
	af1=f1*fbasm^-1
	af2=f2*fbasm^-1
	print "squared angles for f1:"
	print af1
	print "squared angles for f2:"
	print af2
	print "--- checking for compatibility of f1 and f2 ---"
	p=(zz^2-25)*(zz-7)
	L1.<be1>=NumberField(zz^2-af2[0]*af1[0])
	LL1.<w2>=PolynomialRing(L1)
	L2.<be2>=NumberField(w2^2-af2[1]*af1[1])
	LL3.<w3>=PolynomialRing(L2)
	if (w3^2-af2[2]*af1[2]).is_irreducible():
		print "alpha_{3,f1}*alpha_{3,f2} is not in N2(alpha_{1,f1}*alpha_{1,f2},alpha_{2,f1}*alpha_{2,f2})" 
	print "--- end of compatibility check ---"


# check Lemma 6.6
# (x + 5)^15*(x-3)*(x-5)^11*(x^2-17*x +68)
def lem66InterlacingCharacteristicPolynomials():
	R.<x>=PolynomialRing(QQ)
	ll=getCoeffsType2(4,[1,-24,188])
	llshift=[R(list(reversed(l)))(x+1) for l in ll]
	lllist=[list(reversed(l.coefficients(sparse=False))) for l in llshift]
	llred=listInterlaceReduce(lllist,(x + 5)*(x-3)*(x-5)*(x^2-17*x +68))
	return listPowerReduce(llred,(x+5)^14*(x-5)^10)

def checkLemma66():
	R.<x>=PolynomialRing(QQ)
	print "--- Infeasibility of " + str(factor((x + 5)^15*(x-3)*(x-5)^11*(x^2-17*x +68))) + "  ---"
	print ""
	tic = time.time()
	llSeidelred=lem66InterlacingCharacteristicPolynomials()
	toc = time.time()
	print "Time taken to compute interlacing char polys: " + str(toc-tic) + " seconds"
	print ""
	numrows=len(llSeidelred)
	m=matrix(numrows,5,[llSeidelred[j] for j in range(numrows)])
	mults=vector([1,1,1,15,11])
	N.<sqrt17>=NumberField(x^2 - 17)
	NN.<z>=PolynomialRing(N)
	al=(17-sqrt17)/2
	froots=[3,al,17-al,-5,5]
	f=prod([(z-r) for r in froots])
	fbas=[NN(f/(z-r)) for r in froots]
	fbaslist=[list(reversed(l.coefficients(sparse=False))) for l in fbas]
	fbasm=matrix(5,5,[fbaslist[j] for j in range(5)])
	bb=mults*fbasm
	#submatrix of m without f1
	mf1=m[range(10)+range(11,numrows),range(5)]
	print "Matrix A' corresponding to interlacing polynomials without f1:" 
	print mf1 
	print ""
	#Farkas certificate 
	fc1=vector([-256444,0,0,-1143,-190])
	print "Certificate of warranty c for f1:"
	print "c = " + str(fc1)  
	ac =  fc1*mf1.transpose()
	print "A'c >= 0: " + str(isAtLeastZero(ac))
	print "A'c = " + str(ac)
	innerprod = sum([fc1[j]*bb[j] for j in range(5)])
	print "<g,c> < 0: " + str(innerprod < 0)
	print "<g,c> = " + str(innerprod)
	print ""
	f1=vector([m[10,j] for j in range(5)])
	af1=f1*fbasm^-1
	print "squared angles for f1:"
	print af1
	print ""
	print "--- checking for interlacing polynomials compatible with f1 ---"
	p=(z^2-25)
	for i in range(10)+range(11,numrows):
		#print i
		f2=vector([m[i,j] for j in range(5)])
		af2=f2*fbasm^-1
		roots1=(z^2-af2[0]*af1[0]).roots()
		if len(roots1) > 0:
			be1=roots1[0][0]
			roots2=(z^2-af2[2]*af1[2]).roots()
			if len(roots2) > 0:
				be2=roots2[0][0]
				roots3=(z^2-af2[3]*af1[3]).roots()
				if len(roots3) > 0:
					be3=roots3[0][0]
				else:
					continue
			else:
				L2.<be2>=NumberField(z^2-af2[1]*af1[1])
				LL3.<w3>=PolynomialRing(L2)
				roots3=(w3^2-af2[2]*af1[2]).roots()
				if len(roots3) > 0:
					be3=roots3[0][0]
				else:
					continue
		else:
			L1.<be1>=NumberField(z^2-af2[0]*af1[0])
			LL1.<w2>=PolynomialRing(L1)
			roots2=(w2^2-af2[1]*af1[1]).roots()
			if len(roots2) > 0:
				be2=roots2[0][0]
				roots3=(w2^2-af2[2]*af1[2]).roots()
				if len(roots3) > 0:
					be3=roots3[0][0]
				else:
					continue
			else:
				L2.<be2>=NumberField(w2^2-af2[1]*af1[1])
				LL3.<w3>=PolynomialRing(L2)
				roots3=(w3^2-af2[2]*af1[2]).roots()
				if len(roots3) > 0:
					be3=roots3[0][0]
				else:
					continue

		if p(3)*be1+p(al)*be2+p(17-al)*be3 in QQ or p(3)*be1+p(al)*be2-p(17-al)*be3 in QQ or p(3)*be1-p(al)*be2+p(17-al)*be3 in QQ or p(3)*be1-p(al)*be2-p(17-al)*be3 in QQ:  
			print str(factor(R(list(reversed(llSeidelred[i]))))) + " is compatible with f1"
	
	print "--- end of compatibility check ---"
	print ""
	#submatrix of m with only f1 and f2
	mmf1=m[[1,10],range(5)]  
	print "Matrix A'' corresponding to interlacing polynomials f1 and f2:" 
	print mmf1 
	print ""
	#Farkas certificate 
	ffc1=vector([0,0,0,-7,-46]) 
	print "Certificate of infeasibility c:"
	print "c = " + str(ffc1)   
	ac = ffc1*mmf1.transpose()
	print "A''c >= 0: " + str(isAtLeastZero(ac))
	print "A''c = " + str(ac)
	innerprod = sum([ffc1[j]*bb[j] for j in range(5)])
	print "<g,c> < 0: " + str(innerprod < 0)
	print "<g,c> = " + str(innerprod)
		

# check Lemma 6.7
# (x + 5)^15*(x-3)^2*(x-5)^8*(x-7)^2*(x^2-15*x +52)
def lem67InterlacingCharacteristicPolynomials():
	R.<x>=PolynomialRing(QQ)
	ll=getCoeffsType2(5,[1,-30,336])
	llshift=[R(list(reversed(l)))(x+1) for l in ll]
	lllist=[list(reversed(l.coefficients(sparse=False))) for l in llshift]
	llred=listInterlaceReduce(lllist, (x + 5)*(x-3)*(x-5)*(x-7)*(x^2-15*x +52))
	return listPowerReduce(llred,(x+5)^14*(x-3)*(x-5)^7*(x-7))

def checkLemma67():
	R.<x>=PolynomialRing(QQ)
	print "--- Infeasibility of " + str(factor((x + 5)^15*(x-3)^2*(x-5)^8*(x-7)^2*(x^2-15*x +52))) + "  ---"
	print ""
	tic = time.time()
	llSeidelred=lem67InterlacingCharacteristicPolynomials()
	toc = time.time()
	print "Time taken to compute interlacing char polys: " + str(toc-tic) + " seconds"
	print ""
	numrows=len(llSeidelred)
	m=matrix(numrows,6,[llSeidelred[j] for j in range(numrows)])
	mults=vector([1,1,15,2,8,2])
	N.<sqrt17>=NumberField(x^2 - 17)
	al=(15-sqrt17)/2
	NN.<z>=PolynomialRing(N)
	froots=[al,15-al,-5,3,5,7]
	f=prod([(z-r) for r in froots])
	fbas=[NN(f/(z-r)) for r in froots]
	fbaslist=[list(reversed(l.coefficients(sparse=False))) for l in fbas]
	fbasm=matrix(6,6,[fbaslist[j] for j in range(6)])
	bb=mults*fbasm
	#submatrix of m without f1
	mf1=m[range(4)+range(5,numrows),range(6)]
	print "Matrix A' corresponding to interlacing polynomials without f1:" 
	print mf1  
	print ""
	#Farkas certificate 
	fc1=vector([0,0,0,708,507,154])
	print "Certificate of warranty c for f1:"
	print "c = " + str(fc1)   
	ac = fc1*mf1.transpose()
	print "A'c >= 0: " + str(isAtLeastZero(ac))
	print "A'c = " + str(ac)
	innerprod = sum([fc1[j]*bb[j] for j in range(6)])
	print "<g,c> < 0: " + str(innerprod < 0)
	print "<g,c> = " + str(innerprod)
	print ""
	#submatrix of m without f2:
	mf2=m[range(5)+range(6,numrows),range(6)]
	print "Matrix A' corresponding to interlacing polynomials without f2:" 
	print mf2
	print ""
	#Farkas certificate:
	fc2=vector([26302558,0,0,39525,6783,1196])
	print "Certificate of warranty c for f2:"
	print "c = " + str(fc2)    
	ac = fc2*mf2.transpose()
	print "A'c >= 0: " + str(isAtLeastZero(ac))
	print "A'c = " + str(ac)
	innerprod = sum([fc2[j]*bb[j] for j in range(6)])
	print "<g,c> < 0: " + str(innerprod < 0)
	print "<g,c> = " + str(innerprod)
	print ""
	f1=vector([m[4,j] for j in range(6)])
	f2=vector([m[5,j] for j in range(6)])
	af1=f1*fbasm^-1
	af2=f2*fbasm^-1
	p=(z^2-25)*(z-3)*(z-7)
	compat=[]
	print "--- checking for interlacing polynomials compatible with f1 and f2 ---"
	for i in range(numrows):
		f3=vector([m[i,j] for j in range(6)])
		af3=f3*fbasm^-1
		roots1=(z^2-af2[0]*af3[0]).roots()
		if len(roots1) > 0:
			al1=roots1[0][0]
			roots2=(z^2-af2[1]*af3[1]).roots()
			if len(roots2) > 0:
				al2=roots2[0][0]
			else:
				L2.<al2>=NumberField(z^2-af2[1]*af3[1])
		else:
			L1.<al1>=NumberField(z^2-af2[0]*af3[0])
			LL1.<zz>=PolynomialRing(L1)
			roots2=(zz^2-af2[1]*af3[1]).roots()
			if len(roots2) > 0:
				al2=roots2[0][0]
			else:
				L2.<al2>=NumberField(zz^2-af2[1]*af3[1])

		if (p(al)*al1+p(15-al)*al2) in QQ or (p(al)*al1-p(15-al)*al2) in QQ:
			compat+=[i]
			print str(factor(R(list(reversed(llSeidelred[i]))))) + " is compatible with f1 and f2"

	print "--- end of compatibility check ---"
	print ""

	m2=m[compat,range(6)]
	print "Matrix A' corresponding to compatible polynomials:" 
	print m2
	print ""

	nn=vector([0,0,20,7,1,1,0])
	print str(nn) + " times A' equals " + str(bb) + ": " + str(nn*m2 == bb)
	#adding to nn any nonzero vector in the left nullspace of m gives a vector with at least one negative entry
	print "Left kernel of A':" 
	print m2.left_kernel()
	print ""

	f3=vector([m[compat[4],j] for j in range(6)])
	f4=vector([m[compat[5],j] for j in range(6)])
	af3=f3*fbasm^-1
	af4=f4*fbasm^-1
	print "squared angles for f1:"
	print af1
	print "squared angles for f2:"
	print af2
	print "squared angles for f3:"
	print af3
	print "squared angles for f4:"
	print af4


# check Lemma 6.9
# (x + 5)^15*(x-3)*(x-4)*(x-5)^10*(x-9)^2
def lem69InterlacingCharacteristicPolynomials():
	R.<x>=PolynomialRing(QQ)
	ll=getCoeffsType2(4,[1,-20,132])
	llshift=[R(list(reversed(l)))(x+1) for l in ll]
	lllist=[list(reversed(l.coefficients(sparse=False))) for l in llshift]
	llred=listInterlaceReduce(lllist, (x + 5)*(x-3)*(x-4)*(x-5)*(x-9))
	return listPowerReduce(llred,(x+5)^14*(x-5)^9*(x-9))

def checkLemma69():
	R.<x>=PolynomialRing(QQ)
	print "--- Infeasibility of " + str(factor((x + 5)^15*(x-3)*(x-4)*(x-5)^10*(x-9)^2)) + "  ---"
	print ""
	tic = time.time()
	llSeidelred=lem69InterlacingCharacteristicPolynomials()
	toc = time.time()
	print "Time taken to compute interlacing char polys: " + str(toc-tic) + " seconds"
	numrows=len(llSeidelred)
	for i in range(6):
		print factor(R(list(reversed(llSeidelred[i]))))
	m=matrix(numrows,5,[llSeidelred[j] for j in range(numrows)])
	mults=vector([1,1,15,10,2])
	froots=[3,4,-5,5,9]
	f=prod([(x-r) for r in froots])
	fbas=[R(f/(x-r)) for r in froots]
	fbaslist=[list(reversed(l.coefficients(sparse=False))) for l in fbas]
	fbasm=matrix(5,5,[fbaslist[j] for j in range(5)])
	bb=mults*fbasm
	m2=m[[3,0,1,4,5],range(5)]
	print "Matrix A' corresponding to last 5 interlacing characteristic polynomials:" 
	print m2
	print ""
	nn=m2.solve_left(bb)
	print "solution to nA' = " + str(bb) + ":"
	print nn
	nn1=vector([0,25,0,2,2])
	nn2=vector([1,25,0,0,3])
	nn3=vector([0,24,2,0,3])
	print "left kernel of A':"
	print m2.left_kernel()
	print ""
	print "Check that the only nonnegative solutions to nA' = " + str(bb) + " are"
	print str(nn1) + ", " + str(nn2) + ", and " + str(nn3)
	print ""
	f1=vector([m2[0,j] for j in range(5)])
	f2=vector([m2[1,j] for j in range(5)])
	f3=vector([m2[2,j] for j in range(5)])
	f4=vector([m2[3,j] for j in range(5)])
	f5=vector([m2[4,j] for j in range(5)])
	af1=f1*fbasm^-1
	af2=f2*fbasm^-1
	af3=f3*fbasm^-1
	af4=f4*fbasm^-1
	af5=f5*fbasm^-1
	print "squared angles for f2:"
	print af1
	print "squared angles for f3:"
	print af2
	print "squared angles for f4:"
	print af3
	print "squared angles for f5:"
	print af4
	print "squared angles for f6:"
	print af5

# check Lemma 6.8
# (x + 5)^14*(x-5)^9*(x-9)*(x^2-4*x-1)*(x^2-12*x+31)
def lem68InterlacingCharacteristicPolynomials():
	R.<x>=PolynomialRing(QQ)
	ll2=getCoeffsWeaklyType2(6,[1,-31,364])
	llshift2=[R(list(reversed(l)))(x+1) for l in ll2]
	lllist2=[list(reversed(l.coefficients(sparse=False))) for l in llshift2]
	llred2=listInterlaceReduce(lllist2,(x+5)*(x-5)*(x-9)*(x^2-4*x-1)*(x^2-12*x+31))
	return listPowerReduceOddPC(llred2,(x+5)^13*(x-5)^8,scc277,7)

def checkLemma68():
	R.<x>=PolynomialRing(QQ)
	print "--- Infeasibility of " + str(factor((x + 5)^14*(x-5)^9*(x-9)*(x^2-4*x-1)*(x^2-12*x+31))) + "  ---"
	print ""
	tic = time.time()
	llSeidelred2=lem68InterlacingCharacteristicPolynomials()
	toc = time.time()
	print "Time taken to compute interlacing char polys: " + str(toc-tic) + " seconds"
	print ""
	numrows=len(llSeidelred2)
	m=matrix(numrows,7,[llSeidelred2[j] for j in range(numrows)])
	mults=vector([14,9,1,1,1,1,1])
	N.<sqrt5>=NumberField(x^2 - 5)
	al=2-sqrt5
	NN.<z>=PolynomialRing(N)
	froots=[-5,5,9,al,4-al,4+al,8-al]
	f=prod([(z-r) for r in froots])
	fbas=[NN(f/(z-r)) for r in froots]
	fbaslist=[list(reversed(l.coefficients(sparse=False))) for l in fbas]
	fbasm=matrix(7,7,[fbaslist[j] for j in range(7)])
	bb=mults*fbasm
	#submatrix of m without f1
	mf1=m[range(1,numrows),range(7)] 
	print "Matrix A' corresponding to interlacing polynomials without f1:" 
	print mf1
	print ""
	#Farkas certificate 
	fc1=vector([0,0,0,-7130,-5303,-1486,-344])
	print "Certificate of warranty c for f1:"
	print "c = " + str(fc1)    
	ac = fc1*mf1.transpose()
	print "A'c >= 0: " + str(isAtLeastZero(ac))
	print "A'c = " + str(ac)
	
	innerprod = sum([fc1[j]*bb[j] for j in range(7)])
	print "<g,c> < 0: " + str(innerprod < 0)
	print "<g,c> = " + str(innerprod)
	print ""
	f1=vector([m[0,j] for j in range(7)])
	af1=f1*fbasm^-1
	print "squared angles for f1:"
	print af1
	print ""
	print "--- checking for interlacing polynomials compatible with f1 ---"
	p=(z^2-25)
	compat=[]
	for i in range(numrows):
		print str(i+1) + "/" + str(numrows)
		f2=vector([m[i,j] for j in range(7)])
		af2=f2*fbasm^-1
		roots1=(z^2-af1[2]*af2[2]).roots()
		if len(roots1) > 0:
			al1=roots1[0][0]
			roots2=(z^2-af1[3]*af2[3]).roots()
			if len(roots2) > 0:
				al2=roots2[0][0]
				roots3=(z^2-af1[4]*af2[4]).roots()
				if len(roots3) > 0:
					al3=roots3[0][0]
					roots4=(z^2-af1[5]*af2[5]).roots()
					if len(roots4) > 0:
						al4=roots4[0][0]
						roots5=(z^2-af1[6]*af2[6]).roots()
						if len(roots5) > 0:
							al5=roots5[0][0]
						else:
							continue
					else:
						L4.<al4>=NumberField(z^2-af1[5]*af2[5])
						LL4.<z4>=PolynomialRing(L4)
						roots5=(z4^2-af1[6]*af2[6]).roots()
						if len(roots5) > 0:
							al5=roots5[0][0]
						else:
							continue
				else:
					L3.<al3>=NumberField(z^2-af1[4]*af2[4])
					LL3.<z3>=PolynomialRing(L3)
					roots4=(z3^2-af1[5]*af2[5]).roots()
					if len(roots4) > 0:
						al4=roots4[0][0]
						roots5=(z3^2-af1[6]*af2[6]).roots()
						if len(roots5) > 0:
							al5=roots5[0][0]
						else:
							continue
					else:
						L4.<al4>=NumberField(z3^2-af1[5]*af2[5])
						LL4.<z4>=PolynomialRing(L4)
						roots5=(z4^2-af1[6]*af2[6]).roots()
						if len(roots5) > 0:
							al5=roots5[0][0]
						else:
							continue
			else:
				L2.<al2>=NumberField(z^2-af1[3]*af2[3])
				LL2.<z2>=PolynomialRing(L2)
				roots3=(z2^2-af1[4]*af2[4]).roots()
				if len(roots3) > 0:
					al3=roots3[0][0]
					roots4=(z2^2-af1[5]*af2[5]).roots()
					if len(roots4) > 0:
						al4=roots4[0][0]
						roots5=(z2^2-af1[6]*af2[6]).roots()
						if len(roots5) > 0:
							al5=roots5[0][0]
						else:
							continue
					else:
						L4.<al4>=NumberField(z2^2-af1[5]*af2[5])
						LL4.<z4>=PolynomialRing(L4)
						roots5=(z4^2-af1[6]*af2[6]).roots()
						if len(roots5) > 0:
							al5=roots5[0][0]
						else:
							continue
				else:
					L3.<al3>=NumberField(z2^2-af1[4]*af2[4])
					LL3.<z3>=PolynomialRing(L3)
					roots4=(z3^2-af1[5]*af2[5]).roots()
					if len(roots4) > 0:
						al4=roots4[0][0]
						roots5=(z3^2-af1[6]*af2[6]).roots()
						if len(roots5) > 0:
							al5=roots5[0][0]
						else:
							continue
					else:
						L4.<al4>=NumberField(z3^2-af1[5]*af2[5])
						LL4.<z4>=PolynomialRing(L4)
						roots5=(z4^2-af1[6]*af2[6]).roots()
						if len(roots5) > 0:
							al5=roots5[0][0]
						else:
							continue
		else:
			L1.<al1>=NumberField(z^2-af1[2]*af2[2])
			LL1.<z1>=PolynomialRing(L1)
			roots2=(z1^2-af1[3]*af2[3]).roots()
			if len(roots2) >0:
				al2=roots2[0][0]
				roots3=(z1^2-af1[4]*af2[4]).roots()
				if len(roots3) > 0:
					al3=roots3[0][0]
					roots4=(z1^2-af1[5]*af2[5]).roots()
					if len(roots4) > 0:
						al4=roots4[0][0]
						roots5=(z1^2-af1[6]*af2[6]).roots()
						if len(roots5) > 0:
							al5=roots5[0][0]
						else:
							continue
					else:
						L4.<al4>=NumberField(z1^2-af1[5]*af2[5])
						LL4.<z4>=PolynomialRing(L4)
						roots5=(z4^2-af1[6]*af2[6]).roots()
						if len(roots5) > 0:
							al5=roots5[0][0]
						else:
							continue
				else:
					L3.<al3>=NumberField(z^2-af1[4]*af2[4])
					LL3.<z3>=PolynomialRing(L3)
					roots4=(z3^2-af1[5]*af2[5]).roots()
					if len(roots4) > 0:
						al4=roots4[0][0]
						roots5=(z3^2-af1[6]*af2[6]).roots()
						if len(roots5) > 0:
							al5=roots5[0][0]
						else:
							continue
					else:
						L4.<al4>=NumberField(z3^2-af1[5]*af2[5])
						LL4.<z4>=PolynomialRing(L4)
						roots5=(z4^2-af1[6]*af2[6]).roots()
						if len(roots5) > 0:
							al5=roots5[0][0]
						else:
							continue
			else:
				L2.<al2>=NumberField(z1^2-af1[3]*af2[3])
				LL2.<z2>=PolynomialRing(L2)
				roots3=(z2^2-af1[4]*af2[4]).roots()
				if len(roots3) > 0:
					al3=roots3[0][0]
					roots4=(z2^2-af1[5]*af2[5]).roots()
					if len(roots4) > 0:
						al4=roots4[0][0]
						roots5=(z2^2-af1[6]*af2[6]).roots()
						if len(roots5) > 0:
							al5=roots5[0][0]
						else:
							L5.<al5>=NumberField(z2^2-af1[6]*af2[6])
					else:
						L4.<al4>=NumberField(z2^2-af1[5]*af2[5])
						LL4.<z4>=PolynomialRing(L4)
						roots5=(z4^2-af1[6]*af2[6]).roots()
						if len(roots5) > 0:
							al5=roots5[0][0]
						else:
							continue
				else:
					L3.<al3>=NumberField(z2^2-af1[4]*af2[4])
					LL3.<z3>=PolynomialRing(L3)
					roots4=(z3^2-af1[5]*af2[5]).roots()
					if len(roots4) > 0:
						al4=roots4[0][0]
						roots5=(z3^2-af1[6]*af2[6]).roots()
						if len(roots5) > 0:
							al5=roots5[0][0]
						else:
							L5.<al5>=NumberField(z3^2-af1[6]*af2[6])
					else:
						L4.<al4>=NumberField(z3^2-af1[5]*af2[5])
						LL4.<z4>=PolynomialRing(L4)
						roots5=(z4^2-af1[6]*af2[6]).roots()
						if len(roots5) > 0:
							al5=roots5[0][0]
						else:
							continue
		uss={1,-1}
		cp=cartesian_product([uss,uss,uss,uss,uss])
		elmnts=[p(9)*al1,p(al)*al2,p(4-al)*al3,p(4+al)*al4,p(8-al)*al5]
		flag=False
		for mtpl in cp:
			if flag:
				continue
			if sum([mtpl[itr]*elmnts[itr] for itr in range(5)]) in QQ:
				flag = True
		if flag:
		 	compat+=[i]
		 	print str(factor(R(list(reversed(llSeidelred2[i]))))) + " is compatible with f1"

	print "--- end of compatibility check ---"
	print ""
	m2=m[compat,range(7)]
	print "Matrix A' corresponding to compatible polynomials:" 
	print m2
	print ""

	print "Solution to nA' = " + str(bb) + ":"
	print m2.solve_left(bb)
	print ""
	print "Left kernel of A':"
	print m2.left_kernel()


In [11]:
#precompute Seidel char polys classes n=41, e=7
#scc417=SeidelCharPolysMod(41,7)
load("mcP417.sage")


In [12]:
# check Lemma 6.11
# get candidates for 41 lines
def lem611Polynomials():
	R.<x>=PolynomialRing(QQ)
	cand41=getCoeffsWeaklyType2(13,[1,-117,6300])
	cand41shift=[R(list(reversed(l)))(x+1) for l in cand41]
	cand41list=[list(reversed(l.coefficients(sparse=False))) for l in cand41shift]
	return listPowerReduceOddPC(cand41list,(x+5)^25*(x-7)^3,scc417,7)

def checkLemma611():
	R.<x>=PolynomialRing(QQ)
	print "--- Candidates for char poly corresponding to 41 lines ---"
	print ""
	tic = time.time()
	cand41red=lem611Polynomials()
	toc = time.time()
	print "Time taken to compute char polys: " + str(toc-tic) + " seconds"
	print ""
	print "Candidates for char poly:"
	for i in range(len(cand41red)):
		ff=R(list(reversed(cand41red[i])))*(x+5)^25*(x-7)^3
		ms=R(ff/gcd(ff,ff.derivative()))
		print i+1, factor(ff)
	print ""
	cand41=[cand41red[i] for i in [0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20]]
	FarkasCertificates=[
	[-26642928,0,0,-26341,-4390], 
	[32108784,0,0,18832,1713], 
	[-13493,0,0,-62], 
	[0,0,0,-427982,-149845,-31263,-5561],
	[241566984,0,0,99410,11046,1227],
	[0,0,0,26648,10679,2150],
	[0,0,0,38390,15614,2995], 
	[0,0,0,-922821,-295413,-53247,-8185], 
	[356804550,0,0,188720,26961,3851], 
	[0,0,0,0,99745,50373,10481],
	[0,0,0,0,0,19125,14174,5890], 
	[2198761371082,0,0,316965325,29410745,2438199,152389,0], 
	[-3905464984,0,0,-889825,-56453,3,0], 
	[0,0,0,0,-48443,-28265,-8404],
	[1339775890,0,0,659363,82421,10302], 
	[0,0,0,0,0,-61,-67], 
	[2871696,0,0,3493,0], 
	[26404920,0,0,15610,1163,14], 
	[0,0,0,-4544,-2238,-513],
	[0,0,0,-3873,-1613,-180]
	]
	bag=[]
	for i in range(len(cand41)):
		ff=R(list(reversed(cand41[i])))*(x+5)^25*(x-7)^3
		ms=R(ff/gcd(ff,ff.derivative()))
		g=R(ff.derivative()/gcd(ff,ff.derivative()))
		deg=ms.degree()
		f=ms+40*x^(deg-2)
		coeffs=list(reversed(f.coefficients(sparse=False)))
		gf=x^(deg-3)*R([coeffs[2],coeffs[1],coeffs[0]])
		ggf=gf(x-1)
		clist=list(reversed(ggf.coefficients(sparse=False)))
		print "------- Infeasibility of " + str(factor(ff)) + " -------"
		print ""
		tic = time.time()
		ll=getCoeffsType2(deg-1,[clist[0],clist[1],clist[2]])
		llpol=[R(list(reversed(l)))(x+1) for l in ll]
		ll2=[list(reversed(l.coefficients(sparse=False))) for l in llpol]
		bag+=[[factor(ff),listInterlaceReduce(ll2,ms)]]
		toc = time.time()
		print "Time taken to compute interlacing char polys: " + str(toc-tic) + " seconds"
		print ""
		deg=ms.degree()
		numrows=len(bag[i][1])
		m=matrix(numrows,deg,[bag[i][1][j] for j in range(numrows)])
		gg=vector(list(reversed(g.coefficients(sparse=False))))
		print "Coefficient matrix A for interlacing char polys:"
		print m
		print ""
		mm=vector(FarkasCertificates[i])
		print "Certificate of infeasibility c:"
		print "c = " + str(mm)
		ac=mm*m.transpose()
		print "Ac >= 0: " + str(isAtLeastZero(ac))
		print "Ac = " + str(ac)
		innerprod=sum([gg[j]*mm[j] for j in range(deg)])
		print "<g,c> < 0: " + str(innerprod < 0)
		print "<g,c> = " + str(innerprod)
		print ""


# check Lemma 6.12
# (x+5)^25 *(x-7)^9 *(x-9)^4 *(x-11)*(x^2-15*x+48)
def lem612InterlacingCharacteristicPolynomials():
	R.<x>=PolynomialRing(QQ)
	ll=getCoeffsWeaklyType2(5,[1,-42,680])
	llshift=[R(list(reversed(l)))(x+1) for l in ll]
	lllist=[list(reversed(l.coefficients(sparse=False))) for l in llshift]
	llred=listInterlaceReduce(lllist,(x+5)*(x-7)*(x-9)*(x-11)*(x^2-15*x+48))
	return listPowerReduce(llred,(x+5)^24*(x-7)^8*(x-9)^3)


def checkLemma612():
	R.<x>=PolynomialRing(QQ)
	print "--- Infeasibility of " + str(factor((x+5)^25 *(x-7)^9 *(x-9)^4 *(x-11)*(x^2-15*x+48))) + "  ---"
	print ""
	tic = time.time()
	llSeidelred=lem612InterlacingCharacteristicPolynomials()
	toc = time.time()
	print "Time taken to compute interlacing char polys: " + str(toc-tic) + " seconds"
	print ""
	numrows=len(llSeidelred)
	m=matrix(numrows,6,[llSeidelred[j] for j in range(numrows)])
	mults=vector([1,1,1,25,9,4])
	N.<sqrt33>=NumberField(x^2 - 33)
	al=(15-sqrt33)/2
	NN.<z>=PolynomialRing(N)
	froots=[al,15-al,11,-5,7,9]
	f=prod([(z-r) for r in froots])
	fbas=[NN(f/(z-r)) for r in froots]
	fbaslist=[list(reversed(l.coefficients(sparse=False))) for l in fbas]
	fbasm=matrix(6,6,[fbaslist[j] for j in range(6)])
	bb=mults*fbasm
	#submatrix of m without f1
	mf1=m[range(1,numrows),range(6)]
	print "Matrix A' corresponding to interlacing polynomials without f1:" 
	print mf1
	print ""
	#Farkas certificate 
	fc1=vector(list(reversed([113,124,0,0,0,0])))
	print "Certificate of warranty c for f1:"
	print "c = " + str(fc1)  
	ac = fc1*mf1.transpose()
	print "A'c >= 0: " + str(isAtLeastZero(ac))
	print "A'c = " + str(ac)
	innerprod = sum([fc1[j]*bb[j] for j in range(6)])
	print "<g,c> < 0: " + str(innerprod < 0)
	print "<g,c> = " + str(innerprod)
	print ""
	#submatrix of m without f2
	mf2=m[range(3)+range(4,numrows),range(6)]
	print "Matrix A' corresponding to interlacing polynomials without f2:" 
	print mf2
	print ""
	#Farkas certificate 
	fc2=vector(list(reversed([33608, 336081, 3461627, 0, 0, 8817840108])))
	print "Certificate of warranty c for f2:"
	print "c = " + str(fc2) 
	ac = fc2*mf2.transpose()
	print "A'c >= 0: " + str(isAtLeastZero(ac))
	print "A'c = " + str(ac)
	innerproduct = sum([fc2[j]*bb[j] for j in range(6)])
	print "<g,c> < 0: " + str(innerprod < 0)
	print "<g,c> = " + str(innerprod)
	print ""
	#submatrix of m without f1
	mf3=m[range(4)+range(5,numrows),range(6)]
	print "Matrix A' corresponding to interlacing polynomials without f3:" 
	print mf3
	print ""
	#Farkas certificate 
	fc3=vector(list(reversed([1289, 5147, 11405, 0, 0, 0])))
	print "Certificate of warranty c for f3:"
	print "c = " + str(fc3)  
	ac = fc3*mf3.transpose()
	print "A'c >= 0: " + str(isAtLeastZero(ac))
	print "A'c = " + str(ac)
	innerprod = sum([fc3[j]*bb[j] for j in range(6)])
	print "<g,c> < 0: " + str(innerprod < 0)
	print "<g,c> = " + str(innerprod)
	print ""

	p=(z+5)*(z-7)*(z-9)
	f1=vector([m[0,j] for j in range(6)])
	af1=f1*fbasm^-1
	f2=vector([m[3,j] for j in range(6)])
	af2=f2*fbasm^-1
	f3=vector([m[4,j] for j in range(6)])
	af3=f3*fbasm^-1

	print "squared angles for f1:"
	print af1
	print "squared angles for f2:"
	print af2
	print "squared angles for f3:"
	print af3
	print "--- checking for compatibility of f1 and f2 ---"
	p=(z+5)*(z-7)*(z-9)
	L1.<be1>=NumberField(z^2-af2[0]*af1[0])
	LL1.<w2>=PolynomialRing(L1)
	if (w2^2-af2[1]*af1[1]).is_irreducible():
		print "alpha_{2,f1}*alpha_{2,f2} is not in N(alpha_{1,f1}*alpha_{1,f2})"
	print "--- end of compatibility check ---"
	print ""


# check Lemma 6.13
# (x+ 5)^25*(x-3)*(x-7)^6*(x-8)*(x-9)^8
def lem613InterlacingCharacteristicPolynomials():
	R.<x>=PolynomialRing(QQ)
	ll=getCoeffsWeaklyType2(4,[1,-26,240])
	llshift=[R(list(reversed(l)))(x+1) for l in ll]
	lllist=[list(reversed(l.coefficients(sparse=False))) for l in llshift]
	llred=listInterlaceReduce(lllist,(x+5)*(x-3)*(x-7)*(x-8)*(x-9))
	return listPowerReduce(llred,(x+5)^24*(x-7)^5*(x-9)^7)

def checkLemma613():
	R.<x>=PolynomialRing(QQ)
	print "--- Infeasibility of " + str(factor((x+ 5)^25*(x-3)*(x-7)^6*(x-8)*(x-9)^8)) + "  ---"
	print ""
	tic = time.time()
	llSeidelred=lem613InterlacingCharacteristicPolynomials()
	toc = time.time()
	print "Time taken to compute interlacing char polys: " + str(toc-tic) + " seconds"
	print ""
	for i in range(len(llSeidelred)):
		ff=R(list(reversed(llSeidelred[i])))*(x+5)^24*(x-7)^5*(x-9)^7
		ms=R(ff/gcd(ff,ff.derivative()))
		print i+1, factor(ff)
	print ""
	numrows=len(llSeidelred)
	m=matrix(numrows,5,[llSeidelred[j] for j in range(numrows)])
	mults=vector([1,1,25,6,8])
	froots=[3,8,-5,7,9]
	f=prod([(x-r) for r in froots])
	fbas=[R(f/(x-r)) for r in froots]
	fbaslist=[list(reversed(l.coefficients(sparse=False))) for l in fbas]
	fbasm=matrix(5,5,[fbaslist[j] for j in range(5)])
	bb=mults*fbasm
	#submatrix of m without f1
	mf1=m[range(2)+range(3,numrows),range(5)]
	print "Matrix A' corresponding to interlacing polynomials without f1:" 
	print mf1
	print "" 
	#Farkas certificate 
	fc1=vector(list(reversed([-43,-304,0,0,-133896])))
	print "Certificate of warranty c for f1:"
	print "c = " + str(fc1)  
	ac = fc1*mf1.transpose()
	print "A'c >= 0: " + str(isAtLeastZero(ac))
	print "A'c = " + str(ac)
	innerprod = sum([fc1[j]*bb[j] for j in range(5)])
	print "<g,c> < 0: " + str(innerprod < 0)
	print "<g,c> = " + str(innerprod)
	print ""
	#submatrix of m without f2
	mf2=m[range(numrows-1),range(5)]
	print "Matrix A' corresponding to interlacing polynomials without f2:" 
	print mf2
	print "" 
	#Farkas certificate 
	fc2=vector(list(reversed([-1562,-14059,0,0,-6402648])))
	print "Certificate of warranty c for f2:"
	print "c = " + str(fc2)  
	ac = fc2*mf2.transpose()
	print "A'c >= 0: " + str(isAtLeastZero(ac))
	print "A'c = " + str(ac)
	innerprod = sum([fc2[j]*bb[j] for j in range(5)])
	print "<g,c> < 0: " + str(innerprod < 0)
	print "<g,c> = " + str(innerprod)
	print ""
	p=(x+5)*(x-7)*(x-9)
	f1=vector([m[2,j] for j in range(5)])
	af1=f1*fbasm^-1
	f2=vector([m[4,j] for j in range(5)])
	af2=f2*fbasm^-1
	print "squared angles for f1:"
	print af1
	print "squared angles for f2:"
	print af2
	print "--- checking for compatibility of f1 and f2 ---"
	p=(x+5)*(x-7)*(x-9)
	L1.<be1>=NumberField(x^2-af2[0]*af1[0])
	LL1.<w2>=PolynomialRing(L1)
	be2=(w2^2-af2[1]*af1[1]).roots()[0][0]
	print p(3)*be1+p(8)*be2
	print p(3)*be1-p(8)*be2
	print "--- end of compatibility check ---"
	print ""


In [13]:
checkLemma53()

--- Candidates for char poly corresponding to 75 lines ---

Time taken to compute char poly candidates: 0.639142036438 seconds

Candidates for char poly:
1 (x - 18) * (x - 13)^4 * (x - 15)^14 * (x + 5)^56
2 (x - 14) * (x - 17)^2 * (x - 13)^4 * (x - 15)^12 * (x + 5)^56
3 (x - 13)^2 * (x - 15)^15 * (x + 5)^56 * (x^2 - 29*x + 202)
4 (x - 17) * (x - 13)^2 * (x - 15)^14 * (x + 5)^56 * (x^2 - 27*x + 178)
5 (x - 14) * (x - 15)^14 * (x + 5)^56 * (x^2 - 28*x + 191)^2
6 (x - 11) * (x - 15)^16 * (x + 5)^56 * (x^2 - 29*x + 206)
7 (x - 13) * (x - 15)^16 * (x + 5)^56 * (x^2 - 27*x + 174)
8 (x - 10) * (x - 15)^18 * (x + 5)^56

------- Infeasibility of (x - 14) * (x - 17)^2 * (x - 13)^4 * (x - 15)^12 * (x + 5)^56 -------

Time taken to compute interlacing char polys: 0.0254259109497 seconds

Coefficient matrix A for interlacing char polys:
[    1   -54  1080 -9474 30719]
[    1   -54  1080 -9466 30615]
[    1   -54  1080 -9458 30495]

Certificate of infeasibility c:
c = (1876556160, 0, 0, 257776, 1841

In [14]:
checkLemma54()

--- Infeasibility of (x - 18) * (x - 13)^4 * (x - 15)^14 * (x + 5)^56  ---

Time taken to compute interlacing char polys: 0.00995779037476 seconds

Interlacing characteristic polynomials:
1 (x - 13)^3 * (x - 15)^13 * (x + 5)^55 * (x^3 - 41*x^2 + 543*x - 2319)
2 (x - 13)^3 * (x - 15)^13 * (x + 5)^55 * (x^3 - 41*x^2 + 543*x - 2311)
3 (x - 13)^3 * (x - 15)^13 * (x + 5)^55 * (x^3 - 41*x^2 + 543*x - 2303)
4 (x - 17) * (x - 9) * (x - 13)^3 * (x - 15)^14 * (x + 5)^55

Matrix A' corresponding to interlacing polynomials without f1:
[    1   -41   543 -2311]
[    1   -41   543 -2303]
[    1   -41   543 -2295]

Certificate of warranty c:
c = (349511, 0, 0, 151)
Ac >= 0: True
Ac = (550, 1758, 2966)
<g,c> < 0: True
<g,c> = -31230

--- Infeasibility of f1 ---

Time taken to compute interlacing char polys: 3.44359588623 seconds

Interlacing characteristic polynomials:
1 (x - 9) * (x - 13)^3 * (x - 15)^12 * (x + 5)^54 * (x^3 - 42*x^2 + 573*x - 2532)
2 (x - 11) * (x - 13)^3 * (x - 15)^12 * (x + 5)^54 *

In [15]:
checkLemma56()

--- Candidates for char poly corresponding to 95 lines ---

Time taken to compute char poly candidates: 0.611865997314 seconds

Candidates for char poly:
1 (x - 22) * (x - 17)^4 * (x - 19)^15 * (x + 5)^75
2 (x - 18) * (x - 21)^2 * (x - 17)^4 * (x - 19)^13 * (x + 5)^75
3 (x - 17)^2 * (x - 19)^16 * (x + 5)^75 * (x^2 - 37*x + 334)
4 (x - 21) * (x - 17)^2 * (x - 19)^15 * (x + 5)^75 * (x^2 - 35*x + 302)
5 (x - 18) * (x - 19)^15 * (x + 5)^75 * (x^2 - 36*x + 319)^2
6 (x - 15) * (x - 19)^17 * (x + 5)^75 * (x^2 - 37*x + 338)
7 (x - 17) * (x - 19)^17 * (x + 5)^75 * (x^2 - 35*x + 298)
8 (x - 14) * (x - 19)^19 * (x + 5)^75

------- Infeasibility of (x - 18) * (x - 21)^2 * (x - 17)^4 * (x - 19)^13 * (x + 5)^75 -------

Time taken to compute interlacing char polys: 0.0261499881744 seconds

Coefficient matrix A for interlacing char polys:
[     1    -70   1824 -20962  89607]
[     1    -70   1824 -20954  89471]
[     1    -70   1824 -20946  89319]

Certificate of infeasibility c:
c = (14218410144, 0,

In [16]:
checkLemma57()

--- Infeasibility of (x - 22) * (x - 17)^4 * (x - 19)^15 * (x + 5)^75  ---

Time taken to compute interlacing char polys: 0.0140328407288 seconds

Interlacing characteristic polynomials:
1 (x - 17)^3 * (x - 19)^14 * (x + 5)^74 * (x^3 - 53*x^2 + 919*x - 5211)
2 (x - 17)^3 * (x - 19)^14 * (x + 5)^74 * (x^3 - 53*x^2 + 919*x - 5203)
3 (x - 17)^3 * (x - 19)^14 * (x + 5)^74 * (x^3 - 53*x^2 + 919*x - 5195)
4 (x - 21) * (x - 13) * (x - 17)^3 * (x - 19)^15 * (x + 5)^74

Matrix A' corresponding to interlacing polynomials without f1:
[    1   -53   919 -5203]
[    1   -53   919 -5195]
[    1   -53   919 -5187]

Certificate of warranty c:
c = (4776043, 0, 0, 917)
Ac >= 0: True
Ac = (4892, 12228, 19564)
<g,c> < 0: True
<g,c> = -122140

--- Infeasibility of f1 ---

Time taken to compute interlacing char polys: 6.84236001968 seconds

Interlacing characteristic polynomials:
1 (x - 13) * (x - 17)^3 * (x - 19)^13 * (x + 5)^74 * (x^3 - 54*x^2 + 957*x - 5560)
2 (x - 15) * (x - 17)^3 * (x - 19)^13 * (x + 5

In [17]:
checkLemma62()

--- Candidates for char poly corresponding to 29 lines ---

Time taken to compute char poly candidates: 70.7995040417 seconds

Candidates for char poly:
1 (x - 11) * (x - 4) * (x - 5)^12 * (x + 5)^15
2 (x - 3) * (x - 5)^11 * (x + 5)^15 * (x^2 - 17*x + 68)
3 (x - 7) * (x - 3) * (x - 5)^10 * (x + 5)^15 * (x^2 - 15*x + 48)
4 (x - 7) * (x - 4) * (x - 5)^9 * (x + 5)^15 * (x^3 - 19*x^2 + 107*x - 169)
5 (x - 5)^11 * (x + 5)^15 * (x^3 - 20*x^2 + 119*x - 188)
6 (x - 4) * (x - 3) * (x - 9)^2 * (x - 5)^10 * (x + 5)^15
7 (x - 7) * (x - 5)^10 * (x + 5)^15 * (x^3 - 18*x^2 + 93*x - 128)
8 (x - 3) * (x - 7)^2 * (x - 5)^9 * (x + 5)^15 * (x^2 - 13*x + 32)
9 (x - 5)^9 * (x + 5)^15 * (x^5 - 30*x^4 + 344*x^3 - 1874*x^2 + 4823*x - 4672)
10 (x - 5)^10 * (x + 5)^15 * (x^4 - 25*x^3 + 219*x^2 - 779*x + 928)
11 (x - 7)^2 * (x - 3)^2 * (x - 5)^8 * (x + 5)^15 * (x^2 - 15*x + 52)
12 (x - 7) * (x - 5)^8 * (x + 5)^15 * (x^5 - 28*x^4 + 298*x^3 - 1500*x^2 + 3557*x - 3176)
13 (x - 8) * (x - 3) * (x - 5)^8 * (x + 5)^15 *

Time taken to compute interlacing char polys: 0.0732560157776 seconds

Coefficient matrix A for interlacing char polys:
[   1  -17  102 -254  217   15]
[   1  -17  102 -246  153  135]
[   1  -17  102 -246  169   55]
[   1  -17  102 -238   89  255]

Certificate of infeasibility c:
c = (0, 0, 0, 67, 75, 76)
Ac >= 0: True
Ac = (397, 5253, 373, 10109)
<g,c> < 0: True
<g,c> = -5975

------- Infeasibility of (x - 9) * (x - 5)^10 * (x + 5)^15 * (x^3 - 16*x^2 + 75*x - 92) -------

Time taken to compute interlacing char polys: 1.0942800045 seconds

Coefficient matrix A for interlacing char polys:
[   1  -25  222 -822 1121 -369]
[   1  -25  222 -814  993  135]
[   1  -25  222 -814 1009   -9]
[   1  -25  222 -814 1025 -121]
[   1  -25  222 -806  881  495]
[   1  -25  222 -806  897  351]
[   1  -25  222 -806  897  383]
[   1  -25  222 -806  913  239]
[   1  -25  222 -798  769  855]
[   1  -25  222 -798  785  711]
[   1  -25  222 -798  785  743]
[   1  -25  222 -798  785  775]
[   1  -25  222 -798 

Time taken to compute interlacing char polys: 0.583971977234 seconds

Coefficient matrix A for interlacing char polys:
[    1   -26   263 -1300  3175 -3218   465]

Certificate of infeasibility c:
c = (-4921960, 0, 0, -3784, 3, 0, 0)
Ac >= 0: True
Ac = (6765)
<g,c> < 0: True
<g,c> = -47767

------- Infeasibility of (x - 7) * (x - 5)^9 * (x + 5)^15 * (x^4 - 23*x^3 + 183*x^2 - 581*x + 596) -------

Time taken to compute interlacing char polys: 5.37937998772 seconds

Coefficient matrix A for interlacing char polys:
[     1    -30    347  -1924   5079  -5118    301]
[     1    -30    347  -1924   5095  -5310    861]
[     1    -30    347  -1924   5111  -5470   1197]
[     1    -30    347  -1916   4911  -4006  -1995]
[     1    -30    347  -1916   4927  -4198  -1435]
[     1    -30    347  -1916   4927  -4166  -1659]
[     1    -30    347  -1916   4943  -4390   -875]
[     1    -30    347  -1916   4943  -4358  -1099]
[     1    -30    347  -1916   4959  -4582   -315]
[     1    -30    347  -

Time taken to compute interlacing char polys: 21.8634560108 seconds

Coefficient matrix A for interlacing char polys:
[     1    -35    497  -3659  14683 -30241  24371   1295]
[     1    -35    497  -3659  14683 -30209  23987   2415]
[     1    -35    497  -3659  14699 -30481  25507   -385]
[     1    -35    497  -3659  14699 -30449  25123    735]
[     1    -35    497  -3659  14715 -30721  26643  -2065]
[     1    -35    497  -3651  14491 -28561  18035   9975]
[     1    -35    497  -3651  14491 -28529  17651  11095]
[     1    -35    497  -3651  14507 -28833  19555   7175]
[     1    -35    497  -3651  14507 -28801  19171   8295]
[     1    -35    497  -3651  14507 -28769  18787   9415]
[     1    -35    497  -3651  14523 -29105  21075   4375]
[     1    -35    497  -3651  14523 -29073  20691   5495]
[     1    -35    497  -3651  14523 -29041  20307   6615]
[     1    -35    497  -3651  14539 -29345  22211   2695]
[     1    -35    497  -3651  14539 -29313  21827   3815]
[     1    -

In [18]:
checkLemma63()

--- Infeasibility of (x - 7)^2 * (x - 5)^10 * (x + 5)^15 * (x^2 - 11*x + 16)  ---

Time taken to compute interlacing char polys: 0.122490167618 seconds

Matrix A' corresponding to interlacing polynomials without f1:
[   1  -18   96 -134  -41]
[   1  -18   96 -134  -25]
[   1  -18   96 -134   -9]
[   1  -18   96 -134    7]
[   1  -18   96 -126 -113]
[   1  -18   96 -126  -97]
[   1  -18   96 -126  -81]
[   1  -18   96 -126  -65]
[   1  -18   96 -126  -49]
[   1  -18   96 -118 -185]
[   1  -18   96 -118 -169]
[   1  -18   96 -118 -153]
[   1  -18   96 -118 -137]
[   1  -18   96 -118 -121]
[   1  -18   96 -118 -105]
[   1  -18   96 -110 -225]
[   1  -18   96 -110 -209]
[   1  -18   96 -110 -193]
[   1  -18   96 -110 -177]
[   1  -18   96 -110 -161]
[   1  -18   96 -102 -265]
[   1  -18   96 -102 -249]
[   1  -18   96 -102 -233]
[   1  -18   96 -102 -217]
[   1  -18   96  -94 -305]
[   1  -18   96  -94 -289]
[   1  -18   96  -94 -273]
[   1  -18   96  -86 -345]
[   1  -18   96  -86 -329]
[

In [19]:
checkLemma64()

--- Infeasibility of (x - 3) * (x - 7)^2 * (x - 5)^9 * (x + 5)^15 * (x^2 - 13*x + 32)  ---

Time taken to compute interlacing char polys: 0.59640288353 seconds

Matrix A' corresponding to interlacing polynomials without f1:
[   1  -23  186 -630  757   -3]
[   1  -23  186 -630  773  -51]
[   1  -23  186 -630  789  -99]
[   1  -23  186 -630  805 -147]
[   1  -23  186 -622  677  165]
[   1  -23  186 -622  693  117]
[   1  -23  186 -622  709   69]
[   1  -23  186 -622  725   21]
[   1  -23  186 -614  597  365]
[   1  -23  186 -614  613  285]
[   1  -23  186 -614  629  237]
[   1  -23  186 -614  645  189]
[   1  -23  186 -606  533  485]
[   1  -23  186 -606  549  405]
[   1  -23  186 -606  549  437]
[   1  -23  186 -606  565  357]
[   1  -23  186 -598  469  605]
[   1  -23  186 -598  485  525]

Certificate of warranty c for f1:
c = (0, 0, 0, -845, -703, -235)
A'c >= 0: True
A'c = (884, 916, 948, 980, 10884, 10916, 10948, 10980, 13364, 20916, 20948, 20980, 23396, 30948, 23428, 30980, 33428, 

In [20]:
checkLemma65()

--- Infeasibility of (x - 7) * (x - 5)^10 * (x + 5)^15 * (x^3 - 18*x^2 + 93*x - 128)  ---

Time taken to compute interlacing char polys: 1.03546595573 seconds

Matrix A' corresponding to interlacing polynomials without f1:
[   1  -25  222 -830 1121 -137]
[   1  -25  222 -830 1121 -105]
[   1  -25  222 -830 1137 -217]
[   1  -25  222 -830 1153 -329]
[   1  -25  222 -822 1009  223]
[   1  -25  222 -822 1009  255]
[   1  -25  222 -822 1025  143]
[   1  -25  222 -822 1025  175]
[   1  -25  222 -822 1041   31]
[   1  -25  222 -822 1041   63]
[   1  -25  222 -822 1057  -49]
[   1  -25  222 -822 1073 -161]
[   1  -25  222 -814  881  695]
[   1  -25  222 -814  897  615]
[   1  -25  222 -814  913  503]
[   1  -25  222 -814  913  535]
[   1  -25  222 -814  929  423]
[   1  -25  222 -814  929  455]
[   1  -25  222 -814  945  343]
[   1  -25  222 -814  961  231]
[   1  -25  222 -806  785  975]
[   1  -25  222 -806  801  895]
[   1  -25  222 -806  817  815]
[   1  -25  222 -806  833  703]
[   1  -2

In [21]:
checkLemma66()

--- Infeasibility of (x - 3) * (x - 5)^11 * (x + 5)^15 * (x^2 - 17*x + 68)  ---

Time taken to compute interlacing char polys: 0.155001878738 seconds

Matrix A' corresponding to interlacing polynomials without f1:
[   1  -20  122 -244   93]
[   1  -20  122 -236    5]
[   1  -20  122 -236   21]
[   1  -20  122 -236   37]
[   1  -20  122 -236   53]
[   1  -20  122 -236   69]
[   1  -20  122 -228  -35]
[   1  -20  122 -228  -19]
[   1  -20  122 -228   -3]
[   1  -20  122 -228   13]
[   1  -20  122 -220  -75]
[   1  -20  122 -220  -59]
[   1  -20  122 -220  -43]
[   1  -20  122 -220  -27]
[   1  -20  122 -212 -115]
[   1  -20  122 -212  -99]
[   1  -20  122 -212  -83]
[   1  -20  122 -204 -155]
[   1  -20  122 -204 -139]
[   1  -20  122 -196 -195]
[   1  -20  122 -196 -179]
[   1  -20  122 -188 -235]

Certificate of warranty c for f1:
c = (-256444, 0, 0, -1143, -190)
A'c >= 0: True
A'c = (4778, 12354, 9314, 6274, 3234, 194, 10810, 7770, 4730, 1690, 9266, 6226, 3186, 146, 7722, 4682, 1642, 

In [22]:
checkLemma67()

--- Infeasibility of (x - 7)^2 * (x - 3)^2 * (x - 5)^8 * (x + 5)^15 * (x^2 - 15*x + 52)  ---

Time taken to compute interlacing char polys: 0.470178127289 seconds

Matrix A' corresponding to interlacing polynomials without f1:
[   1  -25  226 -890 1405 -525]
[   1  -25  226 -882 1277  -85]
[   1  -25  226 -882 1293 -165]
[   1  -25  226 -882 1309 -245]
[   1  -25  226 -874 1165  275]
[   1  -25  226 -874 1181  195]
[   1  -25  226 -874 1197  115]
[   1  -25  226 -874 1213   35]
[   1  -25  226 -866 1085  475]
[   1  -25  226 -866 1101  395]
[   1  -25  226 -866 1117  315]
[   1  -25  226 -858 1005  675]
[   1  -25  226 -858 1021  595]
[   1  -25  226 -850  925  875]
[   1  -25  226 -842  829 1155]

Certificate of warranty c for f1:
c = (0, 0, 0, 708, 507, 154)
A'c >= 0: True
A'c = (1365, 9893, 5685, 1477, 14213, 10005, 5797, 1589, 10117, 5909, 1701, 6021, 1813, 1925, 2037)
<g,c> < 0: True
<g,c> = -37663

Matrix A' corresponding to interlacing polynomials without f2:
[   1  -25  226 -89

In [23]:
checkLemma68()

--- Infeasibility of (x - 9) * (x - 5)^9 * (x + 5)^14 * (x^2 - 12*x + 31) * (x^2 - 4*x - 1)  ---

Time taken to compute interlacing char polys: 455.624200106 seconds

Matrix A' corresponding to interlacing polynomials without f1:
[     1    -25    224   -838   1013    495   -390]
[     1    -25    224   -838   1013    527   -550]
[     1    -25    224   -838   1013    591   -806]
[     1    -25    224   -834    945    843   -994]
[     1    -25    224   -834    945    875  -1090]
[     1    -25    224   -834    945    939  -1346]
[     1    -25    224   -834    977    491    -98]
[     1    -25    224   -834    977    491    -34]
[     1    -25    224   -834    977    523   -194]
[     1    -25    224   -834    977    555   -354]
[     1    -25    224   -830    877   1207  -1582]
[     1    -25    224   -830    877   1239  -1742]
[     1    -25    224   -830    877   1271  -1838]
[     1    -25    224   -830    909    759   -270]
[     1    -25    224   -830    909    791   -430]
[    

37/124
(x - 5) * (x^2 - 8*x - 1) * (x^3 - 12*x^2 + 29*x + 14) is compatible with f1
38/124
39/124
40/124
41/124
42/124
43/124
44/124
45/124
46/124
47/124
48/124
49/124
50/124
51/124
52/124
53/124
54/124
55/124
56/124
57/124
58/124
59/124
60/124
61/124
62/124
63/124
64/124
65/124
66/124
(x^3 - 14*x^2 + 55*x - 58) * (x^3 - 11*x^2 + 15*x + 59) is compatible with f1
67/124
68/124
69/124
70/124
71/124
72/124
73/124
74/124
75/124
76/124
77/124
78/124
79/124
80/124
81/124
82/124
83/124
84/124
85/124
86/124
87/124
88/124
89/124
90/124
91/124
92/124
93/124
94/124
95/124
96/124
97/124
98/124
99/124
100/124
101/124
102/124
103/124
104/124
105/124
106/124
107/124
108/124
109/124
110/124
111/124
112/124
113/124
114/124
115/124
116/124
117/124
118/124
119/124
120/124
121/124
122/124
123/124
124/124
--- end of compatibility check ---

Matrix A' corresponding to compatible polynomials:
[    1   -25   224  -842  1065   387  -554]
[    1   -25   224  -830   877  1239 -1742]
[    1   -25   224  -826   82

In [24]:
checkLemma69()

--- Infeasibility of (x - 4) * (x - 3) * (x - 9)^2 * (x - 5)^10 * (x + 5)^15  ---

Time taken to compute interlacing char polys: 0.0445809364319 seconds
(x - 3) * (x^3 - 13*x^2 + 39*x - 11)
(x - 3) * (x^3 - 13*x^2 + 39*x - 3)
(x^2 - 12*x + 31) * (x^2 - 4*x - 1)
(x - 5) * (x - 3) * (x^2 - 8*x - 1)
(x - 5) * (x^3 - 11*x^2 + 23*x + 11)
(x - 5) * (x^3 - 11*x^2 + 23*x + 19)
Matrix A' corresponding to last 5 interlacing characteristic polynomials:
[   1  -16   78 -112  -15]
[   1  -16   78 -128   33]
[   1  -16   78 -120    9]
[   1  -16   78 -104  -55]
[   1  -16   78  -96  -95]

solution to nA' = (29, -464, 2262, -3600, 525):
(-2, 25, 0, 6, 0)
left kernel of A':
Vector space of degree 5 and dimension 2 over Rational Field
Basis matrix:
[ 1  0  0 -2  1]
[ 0  1 -2  2 -1]

Check that the only nonnegative solutions to nA' = (29, -464, 2262, -3600, 525) are
(0, 25, 0, 2, 2), (1, 25, 0, 0, 3), and (0, 24, 2, 0, 3)

squared angles for f2:
(0, 17/45, 32/63, 0, 4/35)
squared angles for f3:
(0, 1/45

In [25]:
checkLemma611()

--- Candidates for char poly corresponding to 41 lines ---

Time taken to compute char polys: 407.217815876 seconds

Candidates for char poly:
1 (x - 9)^4 * (x - 7)^10 * (x + 5)^25 * (x^2 - 19*x + 80)
2 (x - 12) * (x - 11) * (x - 9)^2 * (x - 7)^12 * (x + 5)^25
3 (x - 8) * (x - 11)^3 * (x - 7)^12 * (x + 5)^25
4 (x - 9)^3 * (x - 7)^9 * (x + 5)^25 * (x^4 - 35*x^3 + 447*x^2 - 2465*x + 4948)
5 (x - 9)^4 * (x - 7)^9 * (x + 5)^25 * (x^3 - 26*x^2 + 213*x - 544)
6 (x - 11) * (x - 9)^3 * (x - 7)^10 * (x + 5)^25 * (x^2 - 17*x + 64)
7 (x - 11)^2 * (x - 9)^2 * (x - 7)^10 * (x + 5)^25 * (x^2 - 15*x + 52)
8 (x - 11) * (x - 8) * (x - 9)^2 * (x - 7)^8 * (x + 5)^25 * (x^2 - 16*x + 59)^2
9 (x - 9)^5 * (x - 7)^8 * (x + 5)^25 * (x^3 - 24*x^2 + 179*x - 412)
10 (x - 11) * (x - 5) * (x - 9)^4 * (x - 7)^8 * (x + 5)^25 * (x^2 - 17*x + 68)
11 (x - 9)^4 * (x - 7)^7 * (x + 5)^25 * (x^5 - 40*x^4 + 626*x^3 - 4784*x^2 + 17829*x - 25904)
12 (x - 11) * (x - 9)^4 * (x - 7)^9 * (x + 5)^25 * (x^2 - 15*x + 48)
13 (x - 9)^4

Time taken to compute interlacing char polys: 0.516288995743 seconds

Coefficient matrix A for interlacing char polys:
[    1   -35   466 -2934  8669 -9527]
[    1   -35   466 -2926  8525 -8911]
[    1   -35   466 -2926  8541 -9055]
[    1   -35   466 -2926  8541 -9023]
[    1   -35   466 -2926  8557 -9167]
[    1   -35   466 -2926  8557 -9135]
[    1   -35   466 -2918  8381 -8295]
[    1   -35   466 -2918  8397 -8407]
[    1   -35   466 -2918  8413 -8519]
[    1   -35   466 -2918  8429 -8663]
[    1   -35   466 -2918  8429 -8631]
[    1   -35   466 -2918  8445 -8775]
[    1   -35   466 -2910  8269 -7903]
[    1   -35   466 -2910  8285 -8015]
[    1   -35   466 -2910  8301 -8159]
[    1   -35   466 -2910  8301 -8127]
[    1   -35   466 -2910  8317 -8271]
[    1   -35   466 -2902  8141 -7399]
[    1   -35   466 -2902  8157 -7511]
[    1   -35   466 -2902  8173 -7623]
[    1   -35   466 -2902  8189 -7767]
[    1   -35   466 -2894  8029 -7007]
[    1   -35   466 -2894  8045 -7119]
[    1 

Time taken to compute interlacing char polys: 0.23576593399 seconds

Coefficient matrix A for interlacing char polys:
[     1    -35    470  -3010   9129 -10395]
[     1    -35    470  -3002   8985  -9779]
[     1    -35    470  -3002   9001  -9923]
[     1    -35    470  -3002   9001  -9891]
[     1    -35    470  -3002   9017 -10035]
[     1    -35    470  -2994   8857  -9275]
[     1    -35    470  -2994   8873  -9387]
[     1    -35    470  -2994   8889  -9531]
[     1    -35    470  -2986   8729  -8771]
[     1    -35    470  -2986   8745  -8883]
[     1    -35    470  -2986   8761  -9027]
[     1    -35    470  -2978   8617  -8379]
[     1    -35    470  -2970   8489  -7875]

Certificate of infeasibility c:
c = (1339775890, 0, 0, 659363, 82421, 10302)
Ac >= 0: True
Ac = (425279, 177591, 12839, 342503, 177751, 94815, 259727, 94975, 12039, 176951, 12199, 94175, 11399)
<g,c> < 0: True
<g,c> = -387497

------- Infeasibility of (x - 9)^5 * (x - 7)^7 * (x + 5)^25 * (x^4 - 31*x^3 + 347*

In [26]:
checkLemma612()

--- Infeasibility of (x - 11) * (x - 9)^4 * (x - 7)^9 * (x + 5)^25 * (x^2 - 15*x + 48)  ---

Time taken to compute interlacing char polys: 4.20601677895 seconds

Matrix A' corresponding to interlacing polynomials without f1:
[     1    -37    522  -3458  10485 -11033]
[     1    -37    522  -3458  10501 -11209]
[     1    -37    522  -3458  10517 -11385]
[     1    -37    522  -3458  10533 -11529]
[     1    -37    522  -3450  10325 -10241]
[     1    -37    522  -3450  10341 -10417]
[     1    -37    522  -3450  10357 -10593]
[     1    -37    522  -3450  10357 -10561]
[     1    -37    522  -3450  10373 -10737]
[     1    -37    522  -3442  10181  -9625]
[     1    -37    522  -3442  10197  -9801]
[     1    -37    522  -3442  10197  -9769]
[     1    -37    522  -3442  10213  -9945]
[     1    -37    522  -3434  10037  -9009]
[     1    -37    522  -3434  10053  -9153]
[     1    -37    522  -3434  10069  -9297]
[     1    -37    522  -3426   9909  -8505]

Certificate of warranty c 

In [27]:
checkLemma613()

--- Infeasibility of (x - 8) * (x - 3) * (x - 7)^6 * (x - 9)^8 * (x + 5)^25  ---

Time taken to compute interlacing char polys: 0.0668089389801 seconds

1 (x - 3)^2 * (x - 7)^6 * (x - 9)^8 * (x + 5)^24
2 (x - 7)^6 * (x - 9)^7 * (x + 5)^24 * (x^3 - 15*x^2 + 63*x - 73)
3 (x - 7)^5 * (x - 9)^7 * (x + 5)^24 * (x^2 - 12*x + 31) * (x^2 - 10*x + 17)
4 (x - 5) * (x - 7)^6 * (x - 9)^7 * (x + 5)^24 * (x^2 - 10*x + 13)
5 (x - 7)^6 * (x - 9)^7 * (x + 5)^24 * (x^3 - 15*x^2 + 63*x - 57)

Matrix A' corresponding to interlacing polynomials without f1:
[   1  -22  168 -522  567]
[   1  -22  168 -514  511]
[   1  -22  168 -506  455]
[   1  -22  168 -498  399]

Certificate of warranty c for f1:
c = (-133896, 0, 0, -304, -43)
A'c >= 0: True
A'c = (411, 387, 363, 339)
<g,c> < 0: True
<g,c> = -9069

Matrix A' corresponding to interlacing polynomials without f2:
[   1  -22  168 -522  567]
[   1  -22  168 -514  511]
[   1  -22  168 -514  527]
[   1  -22  168 -506  455]

Certificate of warranty c for f2:
c = (